<a href="https://colab.research.google.com/github/LeticiaHms/data-collection-mba/blob/main/limpeza_transformacao_analytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
Script 01: Geração de Dados de Exemplo para Aula
Gera datasets com problemas comuns para exercícios práticos
"""

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

# Configuração de seed para reprodutibilidade
np.random.seed(42)
random.seed(42)

def gerar_dados_vendas_problematicos(n_registros=10000):
    """
    Gera dataset de vendas com problemas típicos:
    - Valores ausentes
    - Duplicatas
    - Outliers
    - Inconsistências de formato
    - Erros de digitação
    """

    # IDs (com algumas duplicatas)
    ids = list(range(1, n_registros + 1))
    # Adicionar duplicatas (5% dos registros)
    n_duplicatas = int(n_registros * 0.05)
    ids_duplicados = random.choices(ids[:n_registros//2], k=n_duplicatas)
    ids.extend(ids_duplicados)

    # Datas
    data_inicio = datetime(2023, 1, 1)
    datas = [data_inicio + timedelta(days=random.randint(0, 365))
             for _ in range(len(ids))]

    # Produtos
    produtos = ['Notebook', 'Mouse', 'Teclado', 'Monitor', 'Webcam',
                'Headset', 'SSD', 'RAM', 'Mousepad', 'Hub USB']
    lista_produtos = [random.choice(produtos) for _ in range(len(ids))]

    # Adicionar inconsistências (maiúsculas/minúsculas, espaços)
    for i in random.sample(range(len(lista_produtos)), len(lista_produtos)//10):
        lista_produtos[i] = lista_produtos[i].lower()
    for i in random.sample(range(len(lista_produtos)), len(lista_produtos)//10):
        lista_produtos[i] = ' ' + lista_produtos[i] + ' '

    # Quantidades
    quantidades = [random.randint(1, 10) for _ in range(len(ids))]

    # Preços (com outliers)
    precos_base = {
        'Notebook': 3500, 'Mouse': 50, 'Teclado': 150, 'Monitor': 800,
        'Webcam': 200, 'Headset': 180, 'SSD': 400, 'RAM': 300,
        'Mousepad': 30, 'Hub USB': 80
    }

    precos = []
    for produto in lista_produtos:
        produto_limpo = produto.strip().title()
        preco_base = precos_base.get(produto_limpo, 100)
        # Variação normal de ±20%
        preco = preco_base * random.uniform(0.8, 1.2)
        precos.append(round(preco, 2))

    # Adicionar outliers (2% dos preços)
    for i in random.sample(range(len(precos)), int(len(precos) * 0.02)):
        precos[i] = precos[i] * random.uniform(10, 50)  # Outliers muito altos

    # Clientes (com variações de formato)
    clientes = []
    for _ in range(len(ids)):
        nome = random.choice(['João Silva', 'Maria Santos', 'Pedro Oliveira',
                             'Ana Costa', 'Carlos Souza', 'Juliana Lima',
                             'Fernando Alves', 'Patricia Rocha'])
        # Variações: maiúscula, minúscula, capitalizada
        formato = random.choice(['normal', 'upper', 'lower'])
        if formato == 'upper':
            nome = nome.upper()
        elif formato == 'lower':
            nome = nome.lower()
        clientes.append(nome)

    # Status (com erros de digitação)
    status_validos = ['Concluído', 'Pendente', 'Cancelado', 'Em Processamento']
    status_com_erros = status_validos + ['Concluido', 'concluído', 'PENDENTE',
                                          'cancelado', 'Em processamento']
    status = [random.choice(status_com_erros) for _ in range(len(ids))]

    # Criar DataFrame
    df = pd.DataFrame({
        'id_venda': ids,
        'data_venda': datas,
        'produto': lista_produtos,
        'quantidade': quantidades,
        'preco_unitario': precos,
        'cliente': clientes,
        'status': status
    })

    # Adicionar valores ausentes (10% em colunas aleatórias)
    colunas_para_missing = ['quantidade', 'preco_unitario', 'cliente', 'status']
    for coluna in colunas_para_missing:
        indices_missing = random.sample(range(len(df)), int(len(df) * 0.1))
        df.loc[indices_missing, coluna] = np.nan

    # Calcular valor total
    df['valor_total'] = df['quantidade'] * df['preco_unitario']

    # Embaralhar registros
    df = df.sample(frac=1).reset_index(drop=True)

    return df


def gerar_dados_sensores_iot(n_registros=500):
    """
    Gera dados de sensores IoT com problemas:
    - Leituras fora do range esperado
    - Timestamps duplicados
    - Gaps temporais
    - Encodings diferentes
    """

    sensor_ids = ['TEMP_01', 'TEMP_02', 'HUMID_01', 'PRESS_01', 'PRESS_02']

    data_inicio = datetime(2024, 1, 1, 0, 0, 0)
    dados = []

    for _ in range(n_registros):
        sensor_id = random.choice(sensor_ids)

        # Timestamp com alguns duplicados
        minutos = random.randint(0, 10000)
        timestamp = data_inicio + timedelta(minutes=minutos)

        # Valores baseados no tipo de sensor
        if 'TEMP' in sensor_id:
            # Temperatura: esperado 15-30°C, outliers 0-50°C
            if random.random() < 0.95:
                valor = random.uniform(15, 30)
            else:
                valor = random.uniform(-10, 50)  # Outlier
        elif 'HUMID' in sensor_id:
            # Umidade: esperado 30-80%, outliers 0-100%
            if random.random() < 0.95:
                valor = random.uniform(30, 80)
            else:
                valor = random.uniform(0, 100)
        else:  # Pressão
            # Pressão: esperado 980-1020 hPa
            if random.random() < 0.95:
                valor = random.uniform(980, 1020)
            else:
                valor = random.uniform(900, 1100)

        # Localização com inconsistências
        localizacoes = ['Sala A', 'sala a', 'SALA A', 'Sala B', 'Armazém 1',
                       'Armazem 1', 'armazém 1']
        localizacao = random.choice(localizacoes)

        dados.append({
            'timestamp': timestamp,
            'sensor_id': sensor_id,
            'valor': round(valor, 2),
            'localizacao': localizacao,
            'unidade': 'C' if 'TEMP' in sensor_id else '%' if 'HUMID' in sensor_id else 'hPa'
        })

    df = pd.DataFrame(dados)

    # Adicionar valores ausentes
    indices_missing = random.sample(range(len(df)), int(len(df) * 0.08))
    df.loc[indices_missing, 'valor'] = np.nan

    # Adicionar alguns registros com timestamp duplicado
    for _ in range(20):
        idx = random.randint(0, len(df)-1)
        df = pd.concat([df, df.iloc[[idx]]], ignore_index=True)

    df = df.sort_values('timestamp').reset_index(drop=True)

    return df


if __name__ == "__main__":
    # Gerar datasets
    print("Gerando datasets de exemplo...")

    # Dataset de vendas
    df_vendas = gerar_dados_vendas_problematicos(1000)
    df_vendas.to_csv('dados_vendas_raw.csv', index=False)
    print(f"✓ dados_vendas_raw.csv criado: {len(df_vendas)} registros")

    # Dataset de sensores IoT
    df_sensores = gerar_dados_sensores_iot(500)
    df_sensores.to_csv('dados_sensores_raw.csv', index=False)
    print(f"✓ dados_sensores_raw.csv criado: {len(df_sensores)} registros")

    # Exibir resumo dos problemas
    print("\n" + "="*60)
    print("PROBLEMAS INSERIDOS - DADOS DE VENDAS:")
    print("="*60)
    print(f"- Duplicatas: ~{df_vendas['id_venda'].duplicated().sum()} registros")
    print(f"- Valores ausentes: {df_vendas.isnull().sum().sum()} células")
    print(f"- Inconsistências de formato em 'produto' e 'cliente'")
    print(f"- Outliers em 'preco_unitario'")
    print(f"- Variações em 'status'")

    print("\n" + "="*60)
    print("PROBLEMAS INSERIDOS - DADOS DE SENSORES:")
    print("="*60)
    print(f"- Timestamps duplicados: ~{df_sensores['timestamp'].duplicated().sum()}")
    print(f"- Valores ausentes: {df_sensores['valor'].isnull().sum()}")
    print(f"- Outliers em leituras dos sensores")
    print(f"- Inconsistências em 'localizacao'")

    print("\n✓ Datasets prontos para os exercícios de pré-processamento!")

Gerando datasets de exemplo...
✓ dados_vendas_raw.csv criado: 1050 registros
✓ dados_sensores_raw.csv criado: 520 registros

PROBLEMAS INSERIDOS - DADOS DE VENDAS:
- Duplicatas: ~50 registros
- Valores ausentes: 619 células
- Inconsistências de formato em 'produto' e 'cliente'
- Outliers em 'preco_unitario'
- Variações em 'status'

PROBLEMAS INSERIDOS - DADOS DE SENSORES:
- Timestamps duplicados: ~36
- Valores ausentes: 45
- Outliers em leituras dos sensores
- Inconsistências em 'localizacao'

✓ Datasets prontos para os exercícios de pré-processamento!


In [ ]:
"""
Script 02: Exploração Inicial e Diagnóstico de Qualidade
Técnicas para identificar problemas nos dados
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Configurar estilo dos gráficos
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)


def analise_completa_dataset(df, nome_dataset="Dataset"):
    """
    Realiza análise exploratória completa do dataset
    """
    print("="*70)
    print(f"ANÁLISE DE QUALIDADE: {nome_dataset}")
    print("="*70)

    # 1. Informações Básicas
    print("\n1. INFORMAÇÕES BÁSICAS")
    print("-" * 70)
    print(f"Dimensões: {df.shape[0]} linhas x {df.shape[1]} colunas")
    print(f"Memória utilizada: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

    # 2. Tipos de Dados
    print("\n2. TIPOS DE DADOS")
    print("-" * 70)
    print(df.dtypes)

    print("\nResumo de tipos:")
    for dtype, count in df.dtypes.value_counts().items():
        print(f"  {dtype}: {count} colunas")

    # 3. Valores Ausentes
    print("\n3. ANÁLISE DE VALORES AUSENTES")
    print("-" * 70)
    missing = df.isnull().sum()
    missing_percent = 100 * missing / len(df)
    missing_df = pd.DataFrame({
        'Coluna': missing.index,
        'Missing': missing.values,
        'Percentual': missing_percent.values
    })
    missing_df = missing_df[missing_df['Missing'] > 0].sort_values('Missing', ascending=False)

    if len(missing_df) > 0:
        print(missing_df.to_string(index=False))
        print(f"\nTotal de células com missing: {missing.sum():,}")
        print(f"Percentual geral de missing: {100 * missing.sum() / (df.shape[0] * df.shape[1]):.2f}%")
    else:
        print("✓ Nenhum valor ausente encontrado")

    # 4. Duplicatas
    print("\n4. ANÁLISE DE DUPLICATAS")
    print("-" * 70)
    duplicatas_completas = df.duplicated().sum()
    print(f"Linhas completamente duplicadas: {duplicatas_completas}")

    if duplicatas_completas > 0:
        print(f"Percentual de duplicatas: {100 * duplicatas_completas / len(df):.2f}%")
        print("\nExemplo de registros duplicados (primeiros 4):")
        print(df[df.duplicated(keep=False)].head(4))

    # 5. Estatísticas Descritivas
    print("\n5. ESTATÍSTICAS DESCRITIVAS")
    print("-" * 70)

    # Numéricas
    colunas_numericas = df.select_dtypes(include=[np.number]).columns
    if len(colunas_numericas) > 0:
        print("\nVariáveis Numéricas:")
        print(df[colunas_numericas].describe())

    # Categóricas
    colunas_categoricas = df.select_dtypes(include=['object']).columns
    if len(colunas_categoricas) > 0:
        print("\nVariáveis Categóricas:")
        for col in colunas_categoricas:
            valores_unicos = df[col].nunique()
            print(f"\n{col}: {valores_unicos} valores únicos")
            if valores_unicos <= 20:
                print(df[col].value_counts().head(10))
            else:
                print(f"(Muitos valores únicos, mostrando top 5)")
                print(df[col].value_counts().head(5))

    # 6. Detecção de Outliers (método IQR)
    print("\n6. DETECÇÃO DE OUTLIERS (Método IQR)")
    print("-" * 70)

    for col in colunas_numericas:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)][col]

        if len(outliers) > 0:
            print(f"\n{col}:")
            print(f"  Range esperado: [{lower_bound:.2f}, {upper_bound:.2f}]")
            print(f"  Outliers encontrados: {len(outliers)} ({100*len(outliers)/len(df):.2f}%)")
            print(f"  Valor mínimo outlier: {outliers.min():.2f}")
            print(f"  Valor máximo outlier: {outliers.max():.2f}")

    return missing_df, duplicatas_completas


def identificar_inconsistencias_categoricas(df, coluna):
    """
    Identifica possíveis inconsistências em colunas categóricas
    (variações de capitalização, espaços, etc.)
    """
    print(f"\nANÁLISE DE INCONSISTÊNCIAS: {coluna}")
    print("-" * 70)

    if coluna not in df.columns:
        print(f"✗ Coluna '{coluna}' não encontrada no dataset")
        return {}

    # Valores únicos
    valores = df[coluna].dropna().unique()
    print(f"Total de valores únicos: {len(valores)}")

    # Agrupar por versão normalizada
    normalizados = {}
    for valor in valores:
        chave = str(valor).strip().lower()
        if chave not in normalizados:
            normalizados[chave] = []
        normalizados[chave].append(valor)

    # Identificar grupos com variações
    print("\nGrupos com variações de formato:")
    inconsistencias_encontradas = False
    for chave, variacoes in normalizados.items():
        if len(variacoes) > 1:
            inconsistencias_encontradas = True
            print(f"\n'{chave}' tem {len(variacoes)} variações:")
            for v in variacoes:
                count = df[df[coluna] == v].shape[0]
                print(f"  - '{v}': {count} ocorrências")

    if not inconsistencias_encontradas:
        print("✓ Nenhuma inconsistência de formato detectada")

    return normalizados


def visualizar_distribuicoes(df, colunas_numericas=None, figsize=(15, 10)):
    """
    Cria visualizações para análise de distribuições e outliers
    """
    if colunas_numericas is None:
        colunas_numericas = df.select_dtypes(include=[np.number]).columns.tolist()

    if len(colunas_numericas) == 0:
        print("Nenhuma coluna numérica para visualizar")
        return

    n_cols = min(3, len(colunas_numericas))
    n_rows = (len(colunas_numericas) + n_cols - 1) // n_cols

    fig, axes = plt.subplots(n_rows, n_cols, figsize=figsize)

    # Garantir que axes seja sempre um array
    if n_rows == 1 and n_cols == 1:
        axes = np.array([axes])
    elif n_rows == 1 or n_cols == 1:
        axes = axes.flatten()
    else:
        axes = axes.flatten()

    for idx, col in enumerate(colunas_numericas):
        ax = axes[idx]

        # Remover NaN para visualização
        data_clean = df[col].dropna()

        if len(data_clean) == 0:
            ax.text(0.5, 0.5, f'{col}\n(Sem dados)',
                   ha='center', va='center', fontsize=12)
            ax.set_xticks([])
            ax.set_yticks([])
            continue

        # Histograma
        ax.hist(data_clean, bins=30, alpha=0.7, color='skyblue', edgecolor='black')
        ax.set_xlabel(col)
        ax.set_ylabel('Frequência')
        ax.set_title(f'Distribuição: {col}')

        # Adicionar linha vertical para média e mediana
        media = data_clean.mean()
        mediana = data_clean.median()
        ax.axvline(media, color='red', linestyle='--', linewidth=2, label=f'Média: {media:.2f}')
        ax.axvline(mediana, color='green', linestyle='--', linewidth=2, label=f'Mediana: {mediana:.2f}')
        ax.legend(fontsize=8)

        # Grid
        ax.grid(True, alpha=0.3)

    # Remover subplots extras
    for idx in range(len(colunas_numericas), len(axes)):
        fig.delaxes(axes[idx])

    plt.tight_layout()
    plt.savefig('distribuicoes_dados.png', dpi=300, bbox_inches='tight')
    print("\n✓ Gráfico salvo: distribuicoes_dados.png")
    plt.close()


def visualizar_boxplots(df, colunas_numericas=None):
    """
    Cria boxplots para identificação visual de outliers
    """
    if colunas_numericas is None:
        colunas_numericas = df.select_dtypes(include=[np.number]).columns.tolist()

    if len(colunas_numericas) == 0:
        print("Nenhuma coluna numérica para visualizar")
        return

    n_cols = min(3, len(colunas_numericas))
    n_rows = (len(colunas_numericas) + n_cols - 1) // n_cols

    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, n_rows * 3))

    # Garantir que axes seja sempre um array
    if n_rows == 1 and n_cols == 1:
        axes = np.array([axes])
    elif n_rows == 1 or n_cols == 1:
        axes = axes.flatten()
    else:
        axes = axes.flatten()

    for idx, col in enumerate(colunas_numericas):
        ax = axes[idx]

        # Remover NaN
        data_clean = df[col].dropna()

        if len(data_clean) == 0:
            ax.text(0.5, 0.5, f'{col}\n(Sem dados)',
                   ha='center', va='center', fontsize=12)
            ax.set_xticks([])
            ax.set_yticks([])
            continue

        # Boxplot
        bp = ax.boxplot(data_clean, vert=True, patch_artist=True)
        bp['boxes'][0].set_facecolor('lightblue')
        bp['boxes'][0].set_edgecolor('black')

        ax.set_ylabel('Valores')
        ax.set_title(f'Boxplot: {col}')
        ax.grid(True, alpha=0.3, axis='y')

    # Remover subplots extras
    for idx in range(len(colunas_numericas), len(axes)):
        fig.delaxes(axes[idx])

    plt.tight_layout()
    plt.savefig('boxplots_dados.png', dpi=300, bbox_inches='tight')
    print("✓ Gráfico salvo: boxplots_dados.png")
    plt.close()


def visualizar_missing_data(df):
    """
    Cria visualização de dados ausentes
    """
    missing_data = df.isnull().sum()
    missing_data = missing_data[missing_data > 0].sort_values(ascending=False)

    if len(missing_data) == 0:
        print("\n✓ Nenhum valor ausente para visualizar")
        return

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

    # Gráfico de barras
    missing_data.plot(kind='bar', ax=ax1, color='coral', edgecolor='black')
    ax1.set_title('Valores Ausentes por Coluna (Contagem)', fontsize=14, fontweight='bold')
    ax1.set_ylabel('Número de Valores Ausentes')
    ax1.set_xlabel('Colunas')
    ax1.grid(True, alpha=0.3, axis='y')
    plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45, ha='right')

    # Gráfico de percentual
    missing_percent = (missing_data / len(df)) * 100
    missing_percent.plot(kind='bar', ax=ax2, color='salmon', edgecolor='black')
    ax2.set_title('Valores Ausentes por Coluna (Percentual)', fontsize=14, fontweight='bold')
    ax2.set_ylabel('Percentual (%)')
    ax2.set_xlabel('Colunas')
    ax2.grid(True, alpha=0.3, axis='y')
    plt.setp(ax2.xaxis.get_majorticklabels(), rotation=45, ha='right')

    plt.tight_layout()
    plt.savefig('missing_data_visualization.png', dpi=300, bbox_inches='tight')
    print("\n✓ Gráfico salvo: missing_data_visualization.png")
    plt.close()


def visualizar_correlacoes(df):
    """
    Cria heatmap de correlação entre variáveis numéricas
    """
    colunas_numericas = df.select_dtypes(include=[np.number]).columns

    if len(colunas_numericas) < 2:
        print("\nInsuficientes colunas numéricas para matriz de correlação")
        return

    # Calcular correlação
    corr_matrix = df[colunas_numericas].corr()

    # Criar heatmap
    plt.figure(figsize=(10, 8))
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0,
                fmt='.2f', square=True, linewidths=1, cbar_kws={"shrink": 0.8})
    plt.title('Matriz de Correlação', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.savefig('correlation_matrix.png', dpi=300, bbox_inches='tight')
    print("\n✓ Gráfico salvo: correlation_matrix.png")
    plt.close()


def gerar_relatorio_qualidade(df, nome_arquivo='relatorio_qualidade.txt'):
    """
    Gera relatório textual completo de qualidade dos dados
    """
    with open(nome_arquivo, 'w', encoding='utf-8') as f:
        f.write("="*70 + "\n")
        f.write("RELATÓRIO DE QUALIDADE DE DADOS\n")
        f.write(f"Gerado em: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write("="*70 + "\n\n")

        # Completude
        f.write("1. COMPLETUDE\n")
        f.write("-"*70 + "\n")
        total_cells = df.shape[0] * df.shape[1]
        missing_cells = df.isnull().sum().sum()
        completude = 100 * (1 - missing_cells / total_cells)
        f.write(f"Completude geral: {completude:.2f}%\n")
        f.write(f"Células com dados: {total_cells - missing_cells:,}\n")
        f.write(f"Células ausentes: {missing_cells:,}\n\n")

        # Completude por coluna
        f.write("Completude por coluna:\n")
        for col in df.columns:
            missing = df[col].isnull().sum()
            comp = 100 * (1 - missing / len(df))
            f.write(f"  {col}: {comp:.2f}%\n")

        f.write("\n")

        # Unicidade
        f.write("2. UNICIDADE\n")
        f.write("-"*70 + "\n")
        duplicatas = df.duplicated().sum()
        unicidade = 100 * (1 - duplicatas / len(df))
        f.write(f"Unicidade: {unicidade:.2f}%\n")
        f.write(f"Registros únicos: {len(df) - duplicatas:,}\n")
        f.write(f"Registros duplicados: {duplicatas:,}\n\n")

        # Consistência
        f.write("3. CONSISTÊNCIA\n")
        f.write("-"*70 + "\n")

        # Verificar tipos de dados
        f.write("Tipos de dados:\n")
        for col, dtype in df.dtypes.items():
            f.write(f"  {col}: {dtype}\n")

        f.write("\n")

        # Validade (ranges)
        f.write("4. VALIDADE - ESTATÍSTICAS\n")
        f.write("-"*70 + "\n")
        colunas_numericas = df.select_dtypes(include=[np.number]).columns
        for col in colunas_numericas:
            f.write(f"\n{col}:\n")
            f.write(f"  Mínimo: {df[col].min()}\n")
            f.write(f"  Máximo: {df[col].max()}\n")
            f.write(f"  Média: {df[col].mean():.2f}\n")
            f.write(f"  Mediana: {df[col].median():.2f}\n")
            f.write(f"  Desvio padrão: {df[col].std():.2f}\n")

    print(f"\n✓ Relatório salvo: {nome_arquivo}")


def gerar_relatorio_html(df, nome_arquivo='relatorio_exploracao.html'):
    """
    Gera relatório HTML interativo
    """
    html_content = f"""
    <!DOCTYPE html>
    <html>
    <head>
        <title>Relatório de Exploração de Dados</title>
        <style>
            body {{ font-family: Arial, sans-serif; margin: 20px; background-color: #f5f5f5; }}
            h1 {{ color: #333; border-bottom: 3px solid #4CAF50; padding-bottom: 10px; }}
            h2 {{ color: #555; margin-top: 30px; }}
            table {{ border-collapse: collapse; width: 100%; margin-top: 20px; background-color: white; }}
            th, td {{ border: 1px solid #ddd; padding: 12px; text-align: left; }}
            th {{ background-color: #4CAF50; color: white; }}
            tr:nth-child(even) {{ background-color: #f2f2f2; }}
            .metric {{ display: inline-block; margin: 10px; padding: 15px; background-color: white;
                     border-radius: 5px; box-shadow: 0 2px 4px rgba(0,0,0,0.1); }}
            .metric-value {{ font-size: 24px; font-weight: bold; color: #4CAF50; }}
            .metric-label {{ font-size: 14px; color: #666; }}
            .warning {{ color: #ff9800; font-weight: bold; }}
            .error {{ color: #f44336; font-weight: bold; }}
            .success {{ color: #4CAF50; font-weight: bold; }}
        </style>
    </head>
    <body>
        <h1>Relatório de Exploração de Dados</h1>
        <p><strong>Gerado em:</strong> {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}</p>

        <h2>Métricas Principais</h2>
        <div class="metric">
            <div class="metric-value">{df.shape[0]:,}</div>
            <div class="metric-label">Linhas</div>
        </div>
        <div class="metric">
            <div class="metric-value">{df.shape[1]}</div>
            <div class="metric-label">Colunas</div>
        </div>
        <div class="metric">
            <div class="metric-value">{100 * (1 - df.isnull().sum().sum() / (df.shape[0] * df.shape[1])):.1f}%</div>
            <div class="metric-label">Completude</div>
        </div>
        <div class="metric">
            <div class="metric-value">{df.duplicated().sum():,}</div>
            <div class="metric-label">Duplicatas</div>
        </div>

        <h2>Estrutura dos Dados</h2>
        <table>
            <tr>
                <th>Coluna</th>
                <th>Tipo</th>
                <th>Valores Únicos</th>
                <th>Missing</th>
                <th>Completude</th>
            </tr>
    """

    for col in df.columns:
        dtype = df[col].dtype
        nunique = df[col].nunique()
        missing = df[col].isnull().sum()
        completude = 100 * (1 - missing / len(df))

        completude_class = 'success' if completude > 95 else 'warning' if completude > 80 else 'error'

        html_content += f"""
            <tr>
                <td><strong>{col}</strong></td>
                <td>{dtype}</td>
                <td>{nunique:,}</td>
                <td>{missing:,}</td>
                <td class="{completude_class}">{completude:.1f}%</td>
            </tr>
        """

    html_content += """
        </table>
    </body>
    </html>
    """

    with open(nome_arquivo, 'w', encoding='utf-8') as f:
        f.write(html_content)

    print(f"\n✓ Relatório HTML salvo: {nome_arquivo}")


if __name__ == "__main__":
    print("="*70)
    print("EXPLORAÇÃO INICIAL DE DADOS")
    print("="*70)

    # Carregar dados de vendas
    try:
        print("\nCarregando dados de vendas...")
        df_vendas = pd.read_csv('dados_vendas_raw.csv')
        print(f"✓ Arquivo carregado: {len(df_vendas)} registros")
    except FileNotFoundError:
        print("\n⚠ Arquivo 'dados_vendas_raw.csv' não encontrado!")
        print("Execute primeiro o script 01_geracao_dados_exemplo.py")
        exit(1)

    # Análise completa
    print("\n" + "="*70)
    missing_info, duplicatas = analise_completa_dataset(df_vendas, "Vendas")

    # Análise de inconsistências em colunas específicas
    if 'produto' in df_vendas.columns:
        identificar_inconsistencias_categoricas(df_vendas, 'produto')

    if 'cliente' in df_vendas.columns:
        identificar_inconsistencias_categoricas(df_vendas, 'cliente')

    if 'status' in df_vendas.columns:
        identificar_inconsistencias_categoricas(df_vendas, 'status')

    # Visualizações
    print("\n" + "="*70)
    print("GERANDO VISUALIZAÇÕES")
    print("="*70)

    colunas_para_visualizar = ['quantidade', 'preco_unitario', 'valor_total']
    # Filtrar apenas colunas que existem
    colunas_existentes = [col for col in colunas_para_visualizar if col in df_vendas.columns]

    if colunas_existentes:
        visualizar_distribuicoes(df_vendas, colunas_existentes)
        visualizar_boxplots(df_vendas, colunas_existentes)

    visualizar_missing_data(df_vendas)
    visualizar_correlacoes(df_vendas)

    # Gerar relatórios
    print("\n" + "="*70)
    print("GERANDO RELATÓRIOS")
    print("="*70)

    gerar_relatorio_qualidade(df_vendas)
    gerar_relatorio_html(df_vendas)

    print("\n" + "="*70)
    print("EXPLORAÇÃO INICIAL CONCLUÍDA!")
    print("="*70)
    print("\nArquivos gerados:")
    print("  • distribuicoes_dados.png")
    print("  • boxplots_dados.png")
    print("  • missing_data_visualization.png")
    print("  • correlation_matrix.png")
    print("  • relatorio_qualidade.txt")
    print("  • relatorio_exploracao.html")
    print("\n✓ Revise os arquivos gerados para identificar problemas nos dados")

In [ ]:
"""
Script 03: Técnicas de Limpeza de Dados
Tratamento de valores ausentes, duplicatas, outliers e inconsistências
"""

import pandas as pd
import numpy as np
from scipy import stats

class LimpadorDados:
    """
    Classe para aplicar técnicas de limpeza de dados
    """

    def __init__(self, df):
        self.df = df.copy()
        self.df_original = df.copy()
        self.log_acoes = []

    def registrar_acao(self, acao):
        """Registra ações de limpeza realizadas"""
        self.log_acoes.append(acao)
        print(f"✓ {acao}")

    # ==================== TRATAMENTO DE VALORES AUSENTES ====================

    def remover_linhas_com_missing(self, threshold=0.5):
        """
        Remove linhas onde mais de 'threshold' das colunas têm valores ausentes
        threshold: proporção de missing aceita (0 a 1)
        """
        linhas_antes = len(self.df)
        missing_por_linha = self.df.isnull().sum(axis=1) / len(self.df.columns)
        self.df = self.df[missing_por_linha <= threshold]
        linhas_removidas = linhas_antes - len(self.df)

        self.registrar_acao(
            f"Removidas {linhas_removidas} linhas com >{threshold*100}% de missing"
        )
        return self

    def remover_colunas_com_missing(self, threshold=0.5):
        """
        Remove colunas onde mais de 'threshold' dos valores são ausentes
        """
        colunas_antes = len(self.df.columns)
        missing_por_coluna = self.df.isnull().sum() / len(self.df)
        colunas_manter = missing_por_coluna[missing_por_coluna <= threshold].index
        self.df = self.df[colunas_manter]
        colunas_removidas = colunas_antes - len(self.df.columns)

        self.registrar_acao(
            f"Removidas {colunas_removidas} colunas com >{threshold*100}% de missing"
        )
        return self

    def imputar_media(self, colunas):
        """Imputa valores ausentes com a média da coluna"""
        for col in colunas:
            if col in self.df.columns:
                missing_antes = self.df[col].isnull().sum()
                media = self.df[col].mean()
                self.df[col].fillna(media, inplace=True)
                self.registrar_acao(
                    f"Imputados {missing_antes} valores em '{col}' com média {media:.2f}"
                )
        return self

    def imputar_mediana(self, colunas):
        """Imputa valores ausentes com a mediana da coluna"""
        for col in colunas:
            if col in self.df.columns:
                missing_antes = self.df[col].isnull().sum()
                mediana = self.df[col].median()
                self.df[col].fillna(mediana, inplace=True)
                self.registrar_acao(
                    f"Imputados {missing_antes} valores em '{col}' com mediana {mediana:.2f}"
                )
        return self

    def imputar_moda(self, colunas):
        """Imputa valores ausentes com a moda (valor mais frequente)"""
        for col in colunas:
            if col in self.df.columns:
                missing_antes = self.df[col].isnull().sum()
                moda = self.df[col].mode()[0] if not self.df[col].mode().empty else None
                if moda is not None:
                    self.df[col].fillna(moda, inplace=True)
                    self.registrar_acao(
                        f"Imputados {missing_antes} valores em '{col}' com moda '{moda}'"
                    )
        return self

    def imputar_forward_fill(self, colunas):
        """Propagação forward (útil para séries temporais)"""
        for col in colunas:
            if col in self.df.columns:
                missing_antes = self.df[col].isnull().sum()
                self.df[col].fillna(method='ffill', inplace=True)
                missing_depois = self.df[col].isnull().sum()
                self.registrar_acao(
                    f"Forward fill em '{col}': {missing_antes - missing_depois} valores preenchidos"
                )
        return self

    def imputar_backward_fill(self, colunas):
        """Propagação backward (útil para séries temporais)"""
        for col in colunas:
            if col in self.df.columns:
                missing_antes = self.df[col].isnull().sum()
                self.df[col].fillna(method='bfill', inplace=True)
                missing_depois = self.df[col].isnull().sum()
                self.registrar_acao(
                    f"Backward fill em '{col}': {missing_antes - missing_depois} valores preenchidos"
                )
        return self

    def imputar_valor_constante(self, coluna, valor):
        """Imputa com valor constante específico"""
        if coluna in self.df.columns:
            missing_antes = self.df[coluna].isnull().sum()
            self.df[coluna].fillna(valor, inplace=True)
            self.registrar_acao(
                f"Imputados {missing_antes} valores em '{coluna}' com constante '{valor}'"
            )
        return self

    # ==================== TRATAMENTO DE DUPLICATAS ====================

    def remover_duplicatas_completas(self, keep='first'):
        """
        Remove linhas completamente duplicadas
        keep: 'first', 'last' ou False (remove todas)
        """
        duplicatas_antes = self.df.duplicated().sum()
        self.df.drop_duplicates(keep=keep, inplace=True)
        self.registrar_acao(
            f"Removidas {duplicatas_antes} linhas duplicadas (keep='{keep}')"
        )
        return self

    def remover_duplicatas_por_chave(self, colunas_chave, keep='first'):
        """Remove duplicatas baseado em colunas específicas"""
        duplicatas_antes = self.df.duplicated(subset=colunas_chave).sum()
        self.df.drop_duplicates(subset=colunas_chave, keep=keep, inplace=True)
        self.registrar_acao(
            f"Removidas {duplicatas_antes} duplicatas baseadas em {colunas_chave}"
        )
        return self

    # ==================== TRATAMENTO DE OUTLIERS ====================

    def remover_outliers_iqr(self, colunas, multiplicador=1.5):
        """
        Remove outliers usando método IQR (Interquartile Range)
        multiplicador: 1.5 (padrão) ou 3.0 (extremos)
        """
        linhas_antes = len(self.df)

        for col in colunas:
            if col in self.df.columns:
                Q1 = self.df[col].quantile(0.25)
                Q3 = self.df[col].quantile(0.75)
                IQR = Q3 - Q1
                lower_bound = Q1 - multiplicador * IQR
                upper_bound = Q3 + multiplicador * IQR

                self.df = self.df[
                    (self.df[col] >= lower_bound) & (self.df[col] <= upper_bound)
                ]

        linhas_removidas = linhas_antes - len(self.df)
        self.registrar_acao(
            f"Removidos {linhas_removidas} outliers (IQR) das colunas {colunas}"
        )
        return self

    def remover_outliers_zscore(self, colunas, threshold=3):
        """
        Remove outliers usando Z-score
        threshold: normalmente 3 (valores além de 3 desvios padrão)
        """
        linhas_antes = len(self.df)

        for col in colunas:
            if col in self.df.columns:
                z_scores = np.abs(stats.zscore(self.df[col].dropna()))
                self.df = self.df[
                    (np.abs(stats.zscore(self.df[col])) < threshold) |
                    self.df[col].isnull()
                ]

        linhas_removidas = linhas_antes - len(self.df)
        self.registrar_acao(
            f"Removidos {linhas_removidas} outliers (Z-score) das colunas {colunas}"
        )
        return self

    def winsorizar(self, colunas, limits=(0.05, 0.05)):
        """
        Winsorização: substitui outliers pelos valores nos percentis
        limits: (lower, upper) - ex: (0.05, 0.05) = 5% inferior e superior
        """
        from scipy.stats.mstats import winsorize

        for col in colunas:
            if col in self.df.columns:
                valores_antes = self.df[col].copy()
                self.df[col] = winsorize(self.df[col].dropna(), limits=limits)
                modificados = (valores_antes != self.df[col]).sum()
                self.registrar_acao(
                    f"Winsorização em '{col}': {modificados} valores ajustados"
                )
        return self

    def cap_outliers(self, coluna, lower_percentile=0.01, upper_percentile=0.99):
        """
        Limita valores aos percentis especificados
        """
        if coluna in self.df.columns:
            lower_bound = self.df[coluna].quantile(lower_percentile)
            upper_bound = self.df[coluna].quantile(upper_percentile)

            modificados = ((self.df[coluna] < lower_bound) |
                          (self.df[coluna] > upper_bound)).sum()

            self.df[coluna] = self.df[coluna].clip(lower=lower_bound, upper=upper_bound)

            self.registrar_acao(
                f"Capping em '{coluna}': {modificados} valores limitados a [{lower_bound:.2f}, {upper_bound:.2f}]"
            )
        return self

    # ==================== PADRONIZAÇÃO E CONSISTÊNCIA ====================

    def padronizar_texto(self, colunas, metodo='title'):
        """
        Padroniza formato de texto
        metodo: 'lower', 'upper', 'title', 'strip'
        """
        for col in colunas:
            if col in self.df.columns:
                if metodo == 'lower':
                    self.df[col] = self.df[col].str.lower()
                elif metodo == 'upper':
                    self.df[col] = self.df[col].str.upper()
                elif metodo == 'title':
                    self.df[col] = self.df[col].str.title()
                elif metodo == 'strip':
                    self.df[col] = self.df[col].str.strip()

                self.registrar_acao(
                    f"Padronização '{metodo}' aplicada em '{col}'"
                )
        return self

    def remover_espacos_extras(self, colunas):
        """Remove espaços em branco extras"""
        for col in colunas:
            if col in self.df.columns:
                self.df[col] = self.df[col].str.strip()
                self.df[col] = self.df[col].str.replace(r'\s+', ' ', regex=True)
                self.registrar_acao(f"Espaços extras removidos em '{col}'")
        return self

    def substituir_valores(self, coluna, mapeamento):
        """
        Substitui valores usando dicionário de mapeamento
        mapeamento: dict com {valor_antigo: valor_novo}
        """
        if coluna in self.df.columns:
            substituicoes = 0
            for old_val, new_val in mapeamento.items():
                count = (self.df[coluna] == old_val).sum()
                substituicoes += count
                self.df[coluna] = self.df[coluna].replace(old_val, new_val)

            self.registrar_acao(
                f"{substituicoes} valores substituídos em '{coluna}'"
            )
        return self

    def normalizar_categorias(self, coluna):
        """
        Normaliza categorias agrupando variações similares
        """
        if coluna in self.df.columns:
            # Criar versão normalizada
            valores_normalizados = self.df[coluna].str.strip().str.lower()

            # Mapear para versão mais comum de cada grupo
            mapeamento = {}
            for valor_norm in valores_normalizados.unique():
                if pd.notna(valor_norm):
                    # Pegar variações originais
                    variacoes = self.df[
                        valores_normalizados == valor_norm
                    ][coluna].dropna().unique()

                    # Escolher a mais frequente como padrão
                    if len(variacoes) > 0:
                        counts = self.df[coluna].value_counts()
                        mais_comum = max(variacoes, key=lambda x: counts.get(x, 0))

                        for variacao in variacoes:
                            if variacao != mais_comum:
                                mapeamento[variacao] = mais_comum

            if mapeamento:
                self.df[coluna] = self.df[coluna].replace(mapeamento)
                self.registrar_acao(
                    f"Normalizadas {len(mapeamento)} variações em '{coluna}'"
                )
        return self

    # ==================== VALIDAÇÃO E CONVERSÃO DE TIPOS ====================

    def converter_tipo(self, coluna, tipo_alvo):
        """
        Converte tipo de dados da coluna
        tipo_alvo: 'int', 'float', 'string', 'datetime', 'category'
        """
        if coluna in self.df.columns:
            try:
                if tipo_alvo == 'int':
                    self.df[coluna] = pd.to_numeric(self.df[coluna], errors='coerce').astype('Int64')
                elif tipo_alvo == 'float':
                    self.df[coluna] = pd.to_numeric(self.df[coluna], errors='coerce')
                elif tipo_alvo == 'string':
                    self.df[coluna] = self.df[coluna].astype(str)
                elif tipo_alvo == 'datetime':
                    self.df[coluna] = pd.to_datetime(self.df[coluna], errors='coerce')
                elif tipo_alvo == 'category':
                    self.df[coluna] = self.df[coluna].astype('category')

                self.registrar_acao(f"Coluna '{coluna}' convertida para {tipo_alvo}")
            except Exception as e:
                print(f"✗ Erro ao converter '{coluna}' para {tipo_alvo}: {str(e)}")
        return self

    def validar_range(self, coluna, min_val, max_val, acao='remover'):
        """
        Valida se valores estão dentro do range esperado
        acao: 'remover', 'clipar', 'marcar'
        """
        if coluna in self.df.columns:
            fora_range = (self.df[coluna] < min_val) | (self.df[coluna] > max_val)
            count_fora = fora_range.sum()

            if acao == 'remover':
                self.df = self.df[~fora_range]
                self.registrar_acao(
                    f"Removidos {count_fora} valores fora do range [{min_val}, {max_val}] em '{coluna}'"
                )
            elif acao == 'clipar':
                self.df[coluna] = self.df[coluna].clip(lower=min_val, upper=max_val)
                self.registrar_acao(
                    f"Clipados {count_fora} valores para range [{min_val}, {max_val}] em '{coluna}'"
                )
            elif acao == 'marcar':
                self.df[f'{coluna}_valido'] = ~fora_range
                self.registrar_acao(
                    f"Marcados {count_fora} valores fora do range em '{coluna}_valido'"
                )
        return self

    # ==================== UTILIDADES ====================

    def resetar(self):
        """Reseta para o dataframe original"""
        self.df = self.df_original.copy()
        self.log_acoes = []
        self.registrar_acao("Dataset resetado para estado original")
        return self

    def obter_df(self):
        """Retorna o dataframe limpo"""
        return self.df

    def exibir_log(self):
        """Exibe todas as ações realizadas"""
        print("\n" + "="*70)
        print("LOG DE LIMPEZA")
        print("="*70)
        for i, acao in enumerate(self.log_acoes, 1):
            print(f"{i}. {acao}")
        print("="*70)

    def comparar_antes_depois(self):
        """Compara estatísticas antes e depois da limpeza"""
        print("\n" + "="*70)
        print("COMPARAÇÃO: ANTES vs DEPOIS DA LIMPEZA")
        print("="*70)

        print(f"\nNúmero de linhas:")
        print(f"  Antes: {len(self.df_original):,}")
        print(f"  Depois: {len(self.df):,}")
        print(f"  Diferença: {len(self.df_original) - len(self.df):,} linhas removidas")

        print(f"\nValores ausentes:")
        print(f"  Antes: {self.df_original.isnull().sum().sum():,}")
        print(f"  Depois: {self.df.isnull().sum().sum():,}")

        print(f"\nDuplicatas:")
        print(f"  Antes: {self.df_original.duplicated().sum():,}")
        print(f"  Depois: {self.df.duplicated().sum():,}")

        print(f"\nMemória:")
        mem_antes = self.df_original.memory_usage(deep=True).sum() / 1024**2
        mem_depois = self.df.memory_usage(deep=True).sum() / 1024**2
        print(f"  Antes: {mem_antes:.2f} MB")
        print(f"  Depois: {mem_depois:.2f} MB")
        print(f"  Economia: {mem_antes - mem_depois:.2f} MB ({100*(mem_antes-mem_depois)/mem_antes:.1f}%)")


# ==================== EXEMPLO DE USO ====================

if __name__ == "__main__":
    # Carregar dados
    print("Carregando dados...")
    df = pd.read_csv('dados_vendas_raw.csv')

    print(f"Dataset original: {df.shape[0]} linhas x {df.shape[1]} colunas")

    # Criar instância do limpador
    limpador = LimpadorDados(df)

    # Pipeline de limpeza
    print("\n" + "="*70)
    print("INICIANDO PIPELINE DE LIMPEZA")
    print("="*70 + "\n")

    df_limpo = (limpador
        # 1. Remover duplicatas completas
        .remover_duplicatas_completas(keep='first')

        # 2. Padronizar textos
        .padronizar_texto(['produto', 'cliente', 'status'], metodo='title')
        .remover_espacos_extras(['produto', 'cliente', 'status'])

        # 3. Normalizar categorias
        .normalizar_categorias('status')

        # 4. Tratar valores ausentes
        .imputar_mediana(['quantidade'])
        .imputar_media(['preco_unitario'])
        .imputar_moda(['status'])

        # 5. Tratar outliers
        .cap_outliers('preco_unitario', lower_percentile=0.01, upper_percentile=0.99)

        # 6. Validar ranges
        .validar_range('quantidade', min_val=0, max_val=100, acao='clipar')

        # 7. Converter tipos
        .converter_tipo('data_venda', 'datetime')

        .obter_df()
    )

    # Exibir resultados
    limpador.exibir_log()
    limpador.comparar_antes_depois()

    # Salvar dados limpos
    df_limpo.to_csv('dados_vendas_limpos.csv', index=False)
    print("\n✓ Dados limpos salvos em: dados_vendas_limpos.csv")

    # Exibir amostra
    print("\n" + "="*70)
    print("AMOSTRA DOS DADOS LIMPOS")
    print("="*70)
    print(df_limpo.head(10))

In [ ]:
"""
Script 04: Transformação de Dados
Normalização, encoding, feature engineering e agregação
"""

import pandas as pd
import numpy as np
from sklearn.preprocessing import (
    StandardScaler, MinMaxScaler, RobustScaler,
    LabelEncoder, OneHotEncoder
)
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings('ignore')


class TransformadorDados:
    """
    Classe para aplicar transformações em dados
    """

    def __init__(self, df):
        self.df = df.copy()
        self.scalers = {}
        self.encoders = {}

    # ==================== NORMALIZAÇÃO E ESCALONAMENTO ====================

    def aplicar_min_max_scaling(self, colunas, feature_range=(0, 1)):
        """
        Min-Max Scaling: escala valores para um range específico
        Formula: X_scaled = (X - X_min) / (X_max - X_min)
        """
        print(f"\n{'='*70}")
        print(f"MIN-MAX SCALING")
        print(f"{'='*70}")

        for col in colunas:
            if col in self.df.columns:
                scaler = MinMaxScaler(feature_range=feature_range)
                valores_originais = self.df[col].values.reshape(-1, 1)
                valores_escalados = scaler.fit_transform(valores_originais)

                self.df[f'{col}_minmax'] = valores_escalados
                self.scalers[f'{col}_minmax'] = scaler

                print(f"✓ {col}:")
                print(f"  Original: [{self.df[col].min():.2f}, {self.df[col].max():.2f}]")
                print(f"  Escalado: [{self.df[f'{col}_minmax'].min():.2f}, {self.df[f'{col}_minmax'].max():.2f}]")

        return self

    def aplicar_standardization(self, colunas):
        """
        Standardization (Z-score): média 0 e desvio padrão 1
        Formula: X_scaled = (X - mean) / std
        """
        print(f"\n{'='*70}")
        print(f"STANDARDIZATION (Z-SCORE)")
        print(f"{'='*70}")

        for col in colunas:
            if col in self.df.columns:
                scaler = StandardScaler()
                valores_originais = self.df[col].values.reshape(-1, 1)
                valores_escalados = scaler.fit_transform(valores_originais)

                self.df[f'{col}_std'] = valores_escalados
                self.scalers[f'{col}_std'] = scaler

                print(f"✓ {col}:")
                print(f"  Média original: {self.df[col].mean():.2f}")
                print(f"  Std original: {self.df[col].std():.2f}")
                print(f"  Média escalada: {self.df[f'{col}_std'].mean():.4f}")
                print(f"  Std escalada: {self.df[f'{col}_std'].std():.4f}")

        return self

    def aplicar_robust_scaling(self, colunas):
        """
        Robust Scaling: usa mediana e IQR (resistente a outliers)
        Formula: X_scaled = (X - median) / IQR
        """
        print(f"\n{'='*70}")
        print(f"ROBUST SCALING")
        print(f"{'='*70}")

        for col in colunas:
            if col in self.df.columns:
                scaler = RobustScaler()
                valores_originais = self.df[col].values.reshape(-1, 1)
                valores_escalados = scaler.fit_transform(valores_originais)

                self.df[f'{col}_robust'] = valores_escalados
                self.scalers[f'{col}_robust'] = scaler

                print(f"✓ {col} escalado com Robust Scaling")

        return self

    def aplicar_log_transform(self, colunas):
        """
        Transformação logarítmica: útil para distribuições assimétricas
        """
        print(f"\n{'='*70}")
        print(f"TRANSFORMAÇÃO LOGARÍTMICA")
        print(f"{'='*70}")

        for col in colunas:
            if col in self.df.columns:
                # Log(x + 1) para evitar log(0)
                self.df[f'{col}_log'] = np.log1p(self.df[col])

                print(f"✓ {col}:")
                print(f"  Assimetria original: {self.df[col].skew():.2f}")
                print(f"  Assimetria log: {self.df[f'{col}_log'].skew():.2f}")

        return self

    # ==================== ENCODING DE VARIÁVEIS CATEGÓRICAS ====================

    def aplicar_label_encoding(self, colunas):
        """
        Label Encoding: converte categorias em números inteiros
        Adequado para variáveis ordinais (com ordem)
        """
        print(f"\n{'='*70}")
        print(f"LABEL ENCODING")
        print(f"{'='*70}")

        for col in colunas:
            if col in self.df.columns:
                le = LabelEncoder()
                self.df[f'{col}_label'] = le.fit_transform(self.df[col].astype(str))
                self.encoders[f'{col}_label'] = le

                print(f"✓ {col}:")
                print(f"  Categorias únicas: {self.df[col].nunique()}")
                print(f"  Mapeamento: {dict(zip(le.classes_, le.transform(le.classes_)))}")

        return self

    def aplicar_onehot_encoding(self, colunas, drop_first=False):
        """
        One-Hot Encoding: cria coluna binária para cada categoria
        Adequado para variáveis nominais (sem ordem)
        """
        print(f"\n{'='*70}")
        print(f"ONE-HOT ENCODING")
        print(f"{'='*70}")

        for col in colunas:
            if col in self.df.columns:
                # Criar dummies
                dummies = pd.get_dummies(
                    self.df[col],
                    prefix=col,
                    drop_first=drop_first
                )

                # Adicionar ao dataframe
                self.df = pd.concat([self.df, dummies], axis=1)

                print(f"✓ {col}:")
                print(f"  Categorias: {self.df[col].nunique()}")
                print(f"  Colunas criadas: {len(dummies.columns)}")
                print(f"  Nomes: {list(dummies.columns)}")

        return self

    def aplicar_frequency_encoding(self, colunas):
        """
        Frequency Encoding: substitui categoria pela sua frequência
        """
        print(f"\n{'='*70}")
        print(f"FREQUENCY ENCODING")
        print(f"{'='*70}")

        for col in colunas:
            if col in self.df.columns:
                freq = self.df[col].value_counts(normalize=True)
                self.df[f'{col}_freq'] = self.df[col].map(freq)

                print(f"✓ {col}: frequências mapeadas")

        return self

    def aplicar_target_encoding(self, coluna_categorica, coluna_target):
        """
        Target Encoding: substitui categoria pela média do target
        Útil para machine learning
        """
        print(f"\n{'='*70}")
        print(f"TARGET ENCODING")
        print(f"{'='*70}")

        if coluna_categorica in self.df.columns and coluna_target in self.df.columns:
            target_means = self.df.groupby(coluna_categorica)[coluna_target].mean()
            self.df[f'{coluna_categorica}_target'] = self.df[coluna_categorica].map(target_means)

            print(f"✓ {coluna_categorica} codificado com médias de {coluna_target}")
            print(f"  Mapeamento:")
            for cat, mean_val in target_means.items():
                print(f"    {cat}: {mean_val:.2f}")

        return self

    # ==================== DISCRETIZAÇÃO (BINNING) ====================

    def aplicar_binning_igual(self, coluna, n_bins, labels=None):
        """
        Binning com intervalos de largura igual
        """
        print(f"\n{'='*70}")
        print(f"BINNING - INTERVALOS IGUAIS")
        print(f"{'='*70}")

        if coluna in self.df.columns:
            self.df[f'{coluna}_bin'] = pd.cut(
                self.df[coluna],
                bins=n_bins,
                labels=labels
            )

            print(f"✓ {coluna} dividido em {n_bins} bins")
            print(f"  Distribuição:")
            print(self.df[f'{coluna}_bin'].value_counts().sort_index())

        return self

    def aplicar_binning_quantil(self, coluna, n_bins, labels=None):
        """
        Binning com bins de tamanho igual (mesmo número de observações)
        """
        print(f"\n{'='*70}")
        print(f"BINNING - QUANTIS")
        print(f"{'='*70}")

        if coluna in self.df.columns:
            self.df[f'{coluna}_qbin'] = pd.qcut(
                self.df[coluna],
                q=n_bins,
                labels=labels,
                duplicates='drop'
            )

            print(f"✓ {coluna} dividido em {n_bins} quantis")
            print(f"  Distribuição:")
            print(self.df[f'{coluna}_qbin'].value_counts().sort_index())

        return self

    def aplicar_binning_customizado(self, coluna, bins, labels):
        """
        Binning com intervalos customizados
        """
        print(f"\n{'='*70}")
        print(f"BINNING - CUSTOMIZADO")
        print(f"{'='*70}")

        if coluna in self.df.columns:
            self.df[f'{coluna}_custom'] = pd.cut(
                self.df[coluna],
                bins=bins,
                labels=labels,
                include_lowest=True
            )

            print(f"✓ {coluna} dividido com bins customizados")
            print(f"  Bins: {bins}")
            print(f"  Labels: {labels}")
            print(f"  Distribuição:")
            print(self.df[f'{coluna}_custom'].value_counts().sort_index())

        return self

    # ==================== FEATURE ENGINEERING ====================

    def criar_features_temporais(self, coluna_data):
        """
        Extrai features de colunas de data/hora
        """
        print(f"\n{'='*70}")
        print(f"FEATURE ENGINEERING - TEMPORAL")
        print(f"{'='*70}")

        if coluna_data in self.df.columns:
            # Garantir que é datetime
            self.df[coluna_data] = pd.to_datetime(self.df[coluna_data])

            # Extrair componentes
            self.df[f'{coluna_data}_ano'] = self.df[coluna_data].dt.year
            self.df[f'{coluna_data}_mes'] = self.df[coluna_data].dt.month
            self.df[f'{coluna_data}_dia'] = self.df[coluna_data].dt.day
            self.df[f'{coluna_data}_dia_semana'] = self.df[coluna_data].dt.dayofweek
            self.df[f'{coluna_data}_dia_ano'] = self.df[coluna_data].dt.dayofyear
            self.df[f'{coluna_data}_semana'] = self.df[coluna_data].dt.isocalendar().week
            self.df[f'{coluna_data}_trimestre'] = self.df[coluna_data].dt.quarter

            # Features booleanas
            self.df[f'{coluna_data}_fim_semana'] = self.df[coluna_data].dt.dayofweek.isin([5, 6])

            print(f"✓ Features temporais criadas para {coluna_data}")
            print(f"  Colunas criadas: ano, mes, dia, dia_semana, dia_ano, semana, trimestre, fim_semana")

        return self

    def criar_features_agregadas(self, grupo_cols, agg_col, agg_funcs):
        """
        Cria features agregadas baseadas em agrupamentos
        """
        print(f"\n{'='*70}")
        print(f"FEATURE ENGINEERING - AGREGAÇÕES")
        print(f"{'='*70}")

        for func in agg_funcs:
            nome_feature = f'{agg_col}_{func}_by_{"_".join(grupo_cols)}'

            if func == 'mean':
                agg_values = self.df.groupby(grupo_cols)[agg_col].transform('mean')
            elif func == 'sum':
                agg_values = self.df.groupby(grupo_cols)[agg_col].transform('sum')
            elif func == 'count':
                agg_values = self.df.groupby(grupo_cols)[agg_col].transform('count')
            elif func == 'std':
                agg_values = self.df.groupby(grupo_cols)[agg_col].transform('std')
            elif func == 'min':
                agg_values = self.df.groupby(grupo_cols)[agg_col].transform('min')
            elif func == 'max':
                agg_values = self.df.groupby(grupo_cols)[agg_col].transform('max')

            self.df[nome_feature] = agg_values
            print(f"✓ {nome_feature}")

        return self

    def criar_features_interacao(self, col1, col2, operacao='multiplicar'):
        """
        Cria features de interação entre duas colunas
        """
        print(f"\n{'='*70}")
        print(f"FEATURE ENGINEERING - INTERAÇÕES")
        print(f"{'='*70}")

        if col1 in self.df.columns and col2 in self.df.columns:
            if operacao == 'multiplicar':
                self.df[f'{col1}_x_{col2}'] = self.df[col1] * self.df[col2]
                print(f"✓ {col1} × {col2}")
            elif operacao == 'dividir':
                self.df[f'{col1}_div_{col2}'] = self.df[col1] / (self.df[col2] + 1e-10)
                print(f"✓ {col1} ÷ {col2}")
            elif operacao == 'somar':
                self.df[f'{col1}_mais_{col2}'] = self.df[col1] + self.df[col2]
                print(f"✓ {col1} + {col2}")
            elif operacao == 'subtrair':
                self.df[f'{col1}_menos_{col2}'] = self.df[col1] - self.df[col2]
                print(f"✓ {col1} - {col2}")

        return self

    def criar_features_polinomiais(self, coluna, grau=2):
        """
        Cria features polinomiais (quadrado, cubo, etc.)
        """
        print(f"\n{'='*70}")
        print(f"FEATURE ENGINEERING - POLINOMIAIS")
        print(f"{'='*70}")

        if coluna in self.df.columns:
            for g in range(2, grau + 1):
                self.df[f'{coluna}_pow{g}'] = self.df[coluna] ** g
                print(f"✓ {coluna}^{g}")

        return self

    # ==================== REDUÇÃO DE DIMENSIONALIDADE ====================

    def aplicar_pca(self, colunas, n_components=2):
        """
        Aplica PCA (Principal Component Analysis)
        """
        print(f"\n{'='*70}")
        print(f"PCA - REDUÇÃO DE DIMENSIONALIDADE")
        print(f"{'='*70}")

        # Verificar se todas as colunas existem
        colunas_validas = [col for col in colunas if col in self.df.columns]

        if len(colunas_validas) > 0:
            # Dados para PCA (sem missing)
            X = self.df[colunas_validas].dropna()

            # Aplicar PCA
            pca = PCA(n_components=n_components)
            componentes = pca.fit_transform(X)

            # Adicionar componentes ao dataframe
            for i in range(n_components):
                self.df.loc[X.index, f'PC{i+1}'] = componentes[:, i]

            # Informações
            print(f"✓ PCA aplicado em {len(colunas_validas)} colunas")
            print(f"  Componentes: {n_components}")
            print(f"  Variância explicada:")
            for i, var in enumerate(pca.explained_variance_ratio_):
                print(f"    PC{i+1}: {var*100:.2f}%")
            print(f"  Variância total explicada: {sum(pca.explained_variance_ratio_)*100:.2f}%")

        return self

    # ==================== AGREGAÇÃO E PIVOTEAMENTO ====================

    def agregar_por_grupo(self, grupo_cols, agg_dict):
        """
        Agrega dados por grupo
        agg_dict: dicionário com {coluna: [funções de agregação]}
        """
        print(f"\n{'='*70}")
        print(f"AGREGAÇÃO POR GRUPO")
        print(f"{'='*70}")

        df_agg = self.df.groupby(grupo_cols).agg(agg_dict).reset_index()

        # Achatar nomes de colunas multi-nível
        df_agg.columns = ['_'.join(col).strip('_') if col[1] else col[0]
                          for col in df_agg.columns.values]

        print(f"✓ Dados agregados por {grupo_cols}")
        print(f"  Shape resultante: {df_agg.shape}")
        print(f"  Colunas: {list(df_agg.columns)}")

        return df_agg

    def criar_pivot_table(self, index, columns, values, aggfunc='mean'):
        """
        Cria tabela pivotada
        """
        print(f"\n{'='*70}")
        print(f"PIVOT TABLE")
        print(f"{'='*70}")

        pivot = pd.pivot_table(
            self.df,
            index=index,
            columns=columns,
            values=values,
            aggfunc=aggfunc,
            fill_value=0
        )

        print(f"✓ Pivot table criada")
        print(f"  Index: {index}")
        print(f"  Columns: {columns}")
        print(f"  Values: {values}")
        print(f"  Shape: {pivot.shape}")

        return pivot

    # ==================== UTILIDADES ====================

    def obter_df(self):
        """Retorna dataframe transformado"""
        return self.df

    def selecionar_colunas(self, colunas):
        """Seleciona apenas colunas específicas"""
        self.df = self.df[colunas]
        print(f"✓ Selecionadas {len(colunas)} colunas")
        return self

    def remover_colunas(self, colunas):
        """Remove colunas específicas"""
        self.df = self.df.drop(columns=colunas, errors='ignore')
        print(f"✓ Removidas colunas: {colunas}")
        return self


# ==================== EXEMPLO DE USO ====================

if __name__ == "__main__":
    # Carregar dados limpos
    print("Carregando dados limpos...")
    df = pd.read_csv('dados_vendas_limpos.csv')

    print(f"\nDataset: {df.shape[0]} linhas x {df.shape[1]} colunas")

    # Criar instância do transformador
    transformador = TransformadorDados(df)

    print("\n" + "="*70)
    print("PIPELINE DE TRANSFORMAÇÃO")
    print("="*70)

    # 1. Features temporais
    transformador.criar_features_temporais('data_venda')

    # 2. Normalização de variáveis numéricas
    transformador.aplicar_standardization(['preco_unitario', 'quantidade'])
    transformador.aplicar_min_max_scaling(['preco_unitario'], feature_range=(0, 1))

    # 3. Transformação logarítmica para dados assimétricos
    transformador.aplicar_log_transform(['valor_total'])

    # 4. Encoding de variáveis categóricas
    transformador.aplicar_label_encoding(['status'])
    transformador.aplicar_onehot_encoding(['produto'], drop_first=True)
    transformador.aplicar_frequency_encoding(['cliente'])

    # 5. Discretização (Binning)
    transformador.aplicar_binning_quantil('preco_unitario', n_bins=4,
                                          labels=['Baixo', 'Médio', 'Alto', 'Premium'])

    # 6. Feature Engineering
    transformador.criar_features_interacao('quantidade', 'preco_unitario', operacao='multiplicar')
    transformador.criar_features_polinomiais('quantidade', grau=2)

    # 7. Features agregadas
    transformador.criar_features_agregadas(
        grupo_cols=['produto'],
        agg_col='valor_total',
        agg_funcs=['mean', 'sum', 'count']
    )

    # Obter dataframe transformado
    df_transformado = transformador.obter_df()

    print("\n" + "="*70)
    print("TRANSFORMAÇÃO CONCLUÍDA")
    print("="*70)
    print(f"Shape final: {df_transformado.shape}")
    print(f"Colunas criadas: {df_transformado.shape[1] - df.shape[1]}")

    # Salvar resultado
    df_transformado.to_csv('dados_vendas_transformados.csv', index=False)
    print("\n✓ Dados transformados salvos em: dados_vendas_transformados.csv")

    # Exibir amostra
    print("\n" + "="*70)
    print("AMOSTRA DOS DADOS TRANSFORMADOS (primeiras 3 linhas)")
    print("="*70)
    print(df_transformado.head(3))

    # ==================== EXEMPLO DE AGREGAÇÃO ====================

    print("\n\n" + "="*70)
    print("EXEMPLO: AGREGAÇÃO DE DADOS")
    print("="*70)

    # Criar relatório de vendas por produto
    relatorio_produto = transformador.agregar_por_grupo(
        grupo_cols=['produto'],
        agg_dict={
            'quantidade': ['sum', 'mean', 'count'],
            'valor_total': ['sum', 'mean', 'min', 'max'],
            'preco_unitario': ['mean']
        }
    )

    print("\nRelatório de Vendas por Produto:")
    print(relatorio_produto)

    # ==================== EXEMPLO DE PIVOT TABLE ====================

    print("\n\n" + "="*70)
    print("EXEMPLO: PIVOT TABLE")
    print("="*70)

    # Criar pivot: vendas por produto e status
    if 'status' in df.columns and 'produto' in df.columns:
        pivot_vendas = transformador.criar_pivot_table(
            index='produto',
            columns='status',
            values='valor_total',
            aggfunc='sum'
        )

        print("\nPivot: Valor Total por Produto e Status")
        print(pivot_vendas)

    print("\n" + "="*70)
    print("✓ PIPELINE COMPLETO EXECUTADO COM SUCESSO!")
    print("="*70)

In [ ]:
"""
Script 05: Validação e Controle de Qualidade de Dados
Sistema completo de regras de validação e métricas de qualidade
"""

import pandas as pd
import numpy as np
import re
from datetime import datetime
from typing import Dict, List, Tuple


class ValidadorQualidade:
    """
    Sistema de validação e controle de qualidade de dados
    """

    def __init__(self, df):
        self.df = df.copy()
        self.resultados_validacao = []
        self.metricas_qualidade = {}

    # ==================== VALIDAÇÕES DE TIPO ====================

    def validar_tipo_coluna(self, coluna, tipo_esperado):
        """
        Valida se coluna tem o tipo de dado esperado
        tipo_esperado: 'int', 'float', 'string', 'datetime', 'bool'
        """
        if coluna not in self.df.columns:
            self._registrar_falha(f"Coluna '{coluna}' não existe")
            return False

        tipo_atual = self.df[coluna].dtype

        mapeamento_tipos = {
            'int': ['int64', 'int32', 'Int64'],
            'float': ['float64', 'float32'],
            'string': ['object', 'string'],
            'datetime': ['datetime64[ns]'],
            'bool': ['bool']
        }

        tipos_validos = mapeamento_tipos.get(tipo_esperado, [])
        validacao_ok = str(tipo_atual) in tipos_validos

        if validacao_ok:
            self._registrar_sucesso(
                f"✓ '{coluna}': tipo correto ({tipo_atual})"
            )
        else:
            self._registrar_falha(
                f"✗ '{coluna}': tipo incorreto. Esperado {tipo_esperado}, encontrado {tipo_atual}"
            )

        return validacao_ok

    # ==================== VALIDAÇÕES DE RANGE ====================

    def validar_range_numerico(self, coluna, min_val=None, max_val=None):
        """
        Valida se valores numéricos estão dentro do range esperado
        """
        if coluna not in self.df.columns:
            self._registrar_falha(f"Coluna '{coluna}' não existe")
            return False

        valores = self.df[coluna].dropna()

        violacoes = 0
        if min_val is not None:
            violacoes += (valores < min_val).sum()
        if max_val is not None:
            violacoes += (valores > max_val).sum()

        if violacoes == 0:
            self._registrar_sucesso(
                f"✓ '{coluna}': todos os valores no range [{min_val}, {max_val}]"
            )
            return True
        else:
            self._registrar_falha(
                f"✗ '{coluna}': {violacoes} valores fora do range [{min_val}, {max_val}]"
            )
            return False

    def validar_valores_positivos(self, coluna):
        """Valida se todos os valores são positivos"""
        return self.validar_range_numerico(coluna, min_val=0)

    def validar_valores_unicos(self, coluna):
        """Valida se todos os valores são únicos (chave primária)"""
        if coluna not in self.df.columns:
            self._registrar_falha(f"Coluna '{coluna}' não existe")
            return False

        duplicatas = self.df[coluna].duplicated().sum()

        if duplicatas == 0:
            self._registrar_sucesso(f"✓ '{coluna}': todos os valores são únicos")
            return True
        else:
            self._registrar_falha(f"✗ '{coluna}': {duplicatas} valores duplicados")
            return False

    # ==================== VALIDAÇÕES DE COMPLETUDE ====================

    def validar_sem_missing(self, colunas):
        """Valida se colunas não têm valores ausentes"""
        resultado = True

        for col in colunas:
            if col not in self.df.columns:
                self._registrar_falha(f"Coluna '{col}' não existe")
                resultado = False
                continue

            missing = self.df[col].isnull().sum()

            if missing == 0:
                self._registrar_sucesso(f"✓ '{col}': sem valores ausentes")
            else:
                self._registrar_falha(
                    f"✗ '{col}': {missing} valores ausentes ({100*missing/len(self.df):.2f}%)"
                )
                resultado = False

        return resultado

    def validar_completude_minima(self, coluna, percentual_minimo=0.95):
        """
        Valida se coluna tem pelo menos X% de completude
        """
        if coluna not in self.df.columns:
            self._registrar_falha(f"Coluna '{coluna}' não existe")
            return False

        completude = 1 - (self.df[coluna].isnull().sum() / len(self.df))

        if completude >= percentual_minimo:
            self._registrar_sucesso(
                f"✓ '{coluna}': completude {completude*100:.2f}% (>= {percentual_minimo*100}%)"
            )
            return True
        else:
            self._registrar_falha(
                f"✗ '{coluna}': completude {completude*100:.2f}% (< {percentual_minimo*100}%)"
            )
            return False

    # ==================== VALIDAÇÕES DE FORMATO ====================

    def validar_formato_regex(self, coluna, padrao, descricao="formato"):
        """
        Valida se valores seguem um padrão regex
        """
        if coluna not in self.df.columns:
            self._registrar_falha(f"Coluna '{coluna}' não existe")
            return False

        valores = self.df[coluna].dropna().astype(str)
        matches = valores.str.match(padrao)
        invalidos = (~matches).sum()

        if invalidos == 0:
            self._registrar_sucesso(f"✓ '{coluna}': todos os valores seguem {descricao}")
            return True
        else:
            self._registrar_falha(
                f"✗ '{coluna}': {invalidos} valores não seguem {descricao}"
            )
            return False

    def validar_email(self, coluna):
        """Valida formato de email"""
        padrao = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
        return self.validar_formato_regex(coluna, padrao, "formato de email")

    def validar_cpf(self, coluna):
        """Valida formato de CPF (XXX.XXX.XXX-XX)"""
        padrao = r'^\d{3}\.\d{3}\.\d{3}-\d{2}$'
        return self.validar_formato_regex(coluna, padrao, "formato de CPF")

    def validar_telefone_br(self, coluna):
        """Valida formato de telefone brasileiro"""
        padrao = r'^\(\d{2}\)\s?\d{4,5}-?\d{4}$'
        return self.validar_formato_regex(coluna, padrao, "formato de telefone")

    def validar_cep(self, coluna):
        """Valida formato de CEP"""
        padrao = r'^\d{5}-?\d{3}$'
        return self.validar_formato_regex(coluna, padrao, "formato de CEP")

    # ==================== VALIDAÇÕES DE NEGÓCIO ====================

    def validar_valores_permitidos(self, coluna, valores_validos):
        """
        Valida se coluna contém apenas valores da lista permitida
        """
        if coluna not in self.df.columns:
            self._registrar_falha(f"Coluna '{coluna}' não existe")
            return False

        valores_invalidos = ~self.df[coluna].isin(valores_validos + [np.nan, None])
        count_invalidos = valores_invalidos.sum()

        if count_invalidos == 0:
            self._registrar_sucesso(
                f"✓ '{coluna}': todos os valores são válidos"
            )
            return True
        else:
            invalidos_unicos = self.df[valores_invalidos][coluna].unique()
            self._registrar_falha(
                f"✗ '{coluna}': {count_invalidos} valores inválidos. "
                f"Encontrados: {list(invalidos_unicos[:5])}"
            )
            return False

    def validar_relacao_colunas(self, col1, col2, operacao, descricao=""):
        """
        Valida relação entre duas colunas
        operacao: '>', '<', '>=', '<=', '==', '!='
        """
        if col1 not in self.df.columns or col2 not in self.df.columns:
            self._registrar_falha(f"Uma ou mais colunas não existem")
            return False

        if operacao == '>':
            violacoes = (self.df[col1] <= self.df[col2]).sum()
        elif operacao == '<':
            violacoes = (self.df[col1] >= self.df[col2]).sum()
        elif operacao == '>=':
            violacoes = (self.df[col1] < self.df[col2]).sum()
        elif operacao == '<=':
            violacoes = (self.df[col1] > self.df[col2]).sum()
        elif operacao == '==':
            violacoes = (self.df[col1] != self.df[col2]).sum()
        elif operacao == '!=':
            violacoes = (self.df[col1] == self.df[col2]).sum()
        else:
            self._registrar_falha(f"Operação '{operacao}' inválida")
            return False

        if violacoes == 0:
            self._registrar_sucesso(
                f"✓ Relação '{col1} {operacao} {col2}': válida {descricao}"
            )
            return True
        else:
            self._registrar_falha(
                f"✗ Relação '{col1} {operacao} {col2}': {violacoes} violações {descricao}"
            )
            return False

    def validar_soma_coluna(self, colunas, coluna_total, tolerancia=0.01):
        """
        Valida se soma de colunas resulta em coluna total
        """
        soma_calculada = self.df[colunas].sum(axis=1)
        diferenca = abs(soma_calculada - self.df[coluna_total])
        violacoes = (diferenca > tolerancia).sum()

        if violacoes == 0:
            self._registrar_sucesso(
                f"✓ Soma de {colunas} = {coluna_total} (tolerância: {tolerancia})"
            )
            return True
        else:
            self._registrar_falha(
                f"✗ {violacoes} inconsistências na soma de {colunas} = {coluna_total}"
            )
            return False

    # ==================== MÉTRICAS DE QUALIDADE ====================

    def calcular_metricas_qualidade(self):
        """
        Calcula métricas abrangentes de qualidade
        """
        print("\n" + "="*70)
        print("MÉTRICAS DE QUALIDADE DE DADOS")
        print("="*70)

        # 1. Completude
        total_cells = self.df.shape[0] * self.df.shape[1]
        missing_cells = self.df.isnull().sum().sum()
        completude = 100 * (1 - missing_cells / total_cells)

        self.metricas_qualidade['completude_geral'] = completude
        print(f"\n1. COMPLETUDE GERAL: {completude:.2f}%")

        # Completude por coluna
        completude_cols = {}
        for col in self.df.columns:
            comp = 100 * (1 - self.df[col].isnull().sum() / len(self.df))
            completude_cols[col] = comp

        self.metricas_qualidade['completude_por_coluna'] = completude_cols

        # 2. Unicidade
        duplicatas = self.df.duplicated().sum()
        unicidade = 100 * (1 - duplicatas / len(self.df))

        self.metricas_qualidade['unicidade'] = unicidade
        print(f"2. UNICIDADE: {unicidade:.2f}%")
        print(f"   Registros duplicados: {duplicatas}")

        # 3. Consistência de tipos
        tipos_inconsistentes = 0
        for col in self.df.columns:
            if self.df[col].dtype == 'object':
                # Verificar se valores podem ser numéricos
                try:
                    pd.to_numeric(self.df[col].dropna(), errors='raise')
                    tipos_inconsistentes += 1
                except:
                    pass

        consistencia_tipos = 100 * (1 - tipos_inconsistentes / len(self.df.columns))
        self.metricas_qualidade['consistencia_tipos'] = consistencia_tipos
        print(f"3. CONSISTÊNCIA DE TIPOS: {consistencia_tipos:.2f}%")

        # 4. Score de Qualidade Geral
        score_qualidade = (completude + unicidade + consistencia_tipos) / 3
        self.metricas_qualidade['score_geral'] = score_qualidade

        print(f"\n{'='*70}")
        print(f"SCORE GERAL DE QUALIDADE: {score_qualidade:.2f}%")
        print(f"{'='*70}")

        # Classificação
        if score_qualidade >= 90:
            classificacao = "EXCELENTE"
        elif score_qualidade >= 75:
            classificacao = "BOM"
        elif score_qualidade >= 60:
            classificacao = "REGULAR"
        else:
            classificacao = "NECESSITA MELHORIAS"

        print(f"Classificação: {classificacao}")

        return self.metricas_qualidade

    # ==================== RELATÓRIOS ====================

    def _registrar_falha(self, mensagem):
        """Registra validação que falhou"""
        self.resultados_validacao.append({
            'status': 'FALHA',
            'mensagem': mensagem,
            'timestamp': datetime.now()
        })

    def gerar_relatorio_validacao(self):
        """
        Gera relatório completo de validação
        """
        print("\n" + "="*70)
        print("RELATÓRIO DE VALIDAÇÃO")
        print("="*70)

        total = len(self.resultados_validacao)
        sucessos = sum(1 for r in self.resultados_validacao if r['status'] == 'SUCESSO')
        falhas = sum(1 for r in self.resultados_validacao if r['status'] == 'FALHA')

        print(f"\nTotal de validações: {total}")
        print(f"Sucessos: {sucessos} ({100*sucessos/total if total > 0 else 0:.1f}%)")
        print(f"Falhas: {falhas} ({100*falhas/total if total > 0 else 0:.1f}%)")

        print("\n" + "-"*70)
        print("DETALHES DAS VALIDAÇÕES")
        print("-"*70)

        for resultado in self.resultados_validacao:
            print(resultado['mensagem'])

        return sucessos, falhas

    def exportar_relatorio(self, nome_arquivo='relatorio_validacao.txt'):
        """
        Exporta relatório de validação para arquivo
        """
        with open(nome_arquivo, 'w', encoding='utf-8') as f:
            f.write("="*70 + "\n")
            f.write("RELATÓRIO DE VALIDAÇÃO DE DADOS\n")
            f.write(f"Gerado em: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
            f.write("="*70 + "\n\n")

            for resultado in self.resultados_validacao:
                f.write(f"[{resultado['status']}] {resultado['mensagem']}\n")

            f.write("\n" + "="*70 + "\n")
            f.write("MÉTRICAS DE QUALIDADE\n")
            f.write("="*70 + "\n")

            for metrica, valor in self.metricas_qualidade.items():
                if isinstance(valor, dict):
                    f.write(f"\n{metrica}:\n")
                    for k, v in valor.items():
                        f.write(f"  {k}: {v:.2f}%\n")
                else:
                    f.write(f"{metrica}: {valor:.2f}%\n")

        print(f"\n✓ Relatório exportado: {nome_arquivo}")


# ==================== FUNÇÕES AUXILIARES ====================

def criar_suite_validacao_vendas(df):
    """
    Cria suite de validação específica para dados de vendas
    """
    validador = ValidadorQualidade(df)

    print("\n" + "="*70)
    print("SUITE DE VALIDAÇÃO: DADOS DE VENDAS")
    print("="*70 + "\n")

    # 1. Validações de tipo
    print("1. VALIDAÇÃO DE TIPOS")
    print("-"*70)
    validador.validar_tipo_coluna('id_venda', 'int')
    validador.validar_tipo_coluna('data_venda', 'datetime')
    validador.validar_tipo_coluna('quantidade', 'int')
    validador.validar_tipo_coluna('preco_unitario', 'float')

    # 2. Validações de completude
    print("\n2. VALIDAÇÃO DE COMPLETUDE")
    print("-"*70)
    validador.validar_sem_missing(['id_venda', 'data_venda', 'produto'])
    validador.validar_completude_minima('preco_unitario', 0.95)
    validador.validar_completude_minima('quantidade', 0.95)

    # 3. Validações de unicidade
    print("\n3. VALIDAÇÃO DE UNICIDADE")
    print("-"*70)
    validador.validar_valores_unicos('id_venda')

    # 4. Validações de range
    print("\n4. VALIDAÇÃO DE RANGES")
    print("-"*70)
    validador.validar_valores_positivos('quantidade')
    validador.validar_valores_positivos('preco_unitario')
    validador.validar_valores_positivos('valor_total')
    validador.validar_range_numerico('quantidade', min_val=1, max_val=100)

    # 5. Validações de negócio
    print("\n5. VALIDAÇÃO DE REGRAS DE NEGÓCIO")
    print("-"*70)

    if 'status' in df.columns:
        status_validos = ['Concluído', 'Pendente', 'Cancelado', 'Em Processamento']
        validador.validar_valores_permitidos('status', status_validos)

    # Validar se valor_total = quantidade * preco_unitario
    if all(col in df.columns for col in ['quantidade', 'preco_unitario', 'valor_total']):
        df_temp = df.dropna(subset=['quantidade', 'preco_unitario', 'valor_total'])
        valor_calculado = df_temp['quantidade'] * df_temp['preco_unitario']
        diferenca = abs(valor_calculado - df_temp['valor_total'])
        violacoes = (diferenca > 0.01).sum()

        if violacoes == 0:
            validador._registrar_sucesso(
                "✓ valor_total = quantidade × preco_unitario"
            )
        else:
            validador._registrar_falha(
                f"✗ {violacoes} inconsistências: valor_total ≠ quantidade × preco_unitario"
            )

    return validador


def criar_suite_validacao_sensores(df):
    """
    Cria suite de validação para dados de sensores IoT
    """
    validador = ValidadorQualidade(df)

    print("\n" + "="*70)
    print("SUITE DE VALIDAÇÃO: DADOS DE SENSORES IOT")
    print("="*70 + "\n")

    # 1. Validações básicas
    print("1. VALIDAÇÃO BÁSICA")
    print("-"*70)
    validador.validar_sem_missing(['timestamp', 'sensor_id', 'unidade'])
    validador.validar_tipo_coluna('timestamp', 'datetime')

    # 2. Validações de range por tipo de sensor
    print("\n2. VALIDAÇÃO DE RANGES POR SENSOR")
    print("-"*70)

    if 'sensor_id' in df.columns and 'valor' in df.columns:
        # Temperatura: -10 a 50°C
        df_temp = df[df['sensor_id'].str.contains('TEMP', na=False)]
        if len(df_temp) > 0:
            fora_range = ((df_temp['valor'] < -10) | (df_temp['valor'] > 50)).sum()
            if fora_range == 0:
                validador._registrar_sucesso(
                    "✓ Sensores TEMP: valores no range [-10°C, 50°C]"
                )
            else:
                validador._registrar_falha(
                    f"✗ Sensores TEMP: {fora_range} valores fora do range [-10°C, 50°C]"
                )

        # Umidade: 0 a 100%
        df_humid = df[df['sensor_id'].str.contains('HUMID', na=False)]
        if len(df_humid) > 0:
            fora_range = ((df_humid['valor'] < 0) | (df_humid['valor'] > 100)).sum()
            if fora_range == 0:
                validador._registrar_sucesso(
                    "✓ Sensores HUMID: valores no range [0%, 100%]"
                )
            else:
                validador._registrar_falha(
                    f"✗ Sensores HUMID: {fora_range} valores fora do range [0%, 100%]"
                )

        # Pressão: 900 a 1100 hPa
        df_press = df[df['sensor_id'].str.contains('PRESS', na=False)]
        if len(df_press) > 0:
            fora_range = ((df_press['valor'] < 900) | (df_press['valor'] > 1100)).sum()
            if fora_range == 0:
                validador._registrar_sucesso(
                    "✓ Sensores PRESS: valores no range [900 hPa, 1100 hPa]"
                )
            else:
                validador._registrar_falha(
                    f"✗ Sensores PRESS: {fora_range} valores fora do range [900 hPa, 1100 hPa]"
                )

    # 3. Validação de duplicatas temporais
    print("\n3. VALIDAÇÃO DE TIMESTAMPS")
    print("-"*70)
    if 'timestamp' in df.columns and 'sensor_id' in df.columns:
        duplicatas = df.duplicated(subset=['timestamp', 'sensor_id']).sum()
        if duplicatas == 0:
            validador._registrar_sucesso(
                "✓ Sem duplicatas de timestamp por sensor"
            )
        else:
            validador._registrar_falha(
                f"✗ {duplicatas} duplicatas de timestamp por sensor"
            )

    return validador


# ==================== EXEMPLO DE USO ====================

if __name__ == "__main__":
    # Carregar dados
    print("Carregando dados limpos...")
    df_vendas = pd.read_csv('dados_vendas_limpos.csv')

    # Converter data_venda para datetime se necessário
    if 'data_venda' in df_vendas.columns:
        df_vendas['data_venda'] = pd.to_datetime(df_vendas['data_venda'])

    # Executar suite de validação
    validador = criar_suite_validacao_vendas(df_vendas)

    # Calcular métricas de qualidade
    metricas = validador.calcular_metricas_qualidade()

    # Gerar relatório
    sucessos, falhas = validador.gerar_relatorio_validacao()

    # Exportar relatório
    validador.exportar_relatorio('relatorio_validacao_vendas.txt')

    # ==================== VALIDAÇÃO DE SENSORES ====================

    print("\n\n" + "="*70)
    print("VALIDAÇÃO DE DADOS DE SENSORES")
    print("="*70)

    try:
        df_sensores = pd.read_csv('dados_sensores_raw.csv')
        df_sensores['timestamp'] = pd.to_datetime(df_sensores['timestamp'])

        validador_sensores = criar_suite_validacao_sensores(df_sensores)
        validador_sensores.calcular_metricas_qualidade()
        validador_sensores.gerar_relatorio_validacao()
        validador_sensores.exportar_relatorio('relatorio_validacao_sensores.txt')
    except FileNotFoundError:
        print("Arquivo de sensores não encontrado, pulando validação...")

    print("\n" + "="*70)
    print("✓ VALIDAÇÃO COMPLETA")
    print("="*70)

In [ ]:
"""
Script 06: Pipeline Completo de Pré-processamento
Integra todas as etapas: exploração, limpeza, transformação e validação
"""

import pandas as pd
import numpy as np
from datetime import datetime
import json
import os


class PipelinePreProcessamento:
    """
    Pipeline completo e configurável de pré-processamento de dados
    """

    def __init__(self, nome_pipeline="Pipeline"):
        self.nome_pipeline = nome_pipeline
        self.df_original = None
        self.df_processado = None
        self.etapas_executadas = []
        self.metricas = {}
        self.tempo_inicio = None
        self.tempo_fim = None

    def carregar_dados(self, caminho_arquivo, **kwargs):
        """
        Carrega dados de arquivo CSV
        """
        print(f"\n{'='*70}")
        print(f"PIPELINE: {self.nome_pipeline}")
        print(f"{'='*70}\n")
        print(f"📁 Carregando dados de: {caminho_arquivo}")

        self.tempo_inicio = datetime.now()

        try:
            self.df_original = pd.read_csv(caminho_arquivo, **kwargs)
            self.df_processado = self.df_original.copy()

            print(f"✓ Dados carregados com sucesso!")
            print(f"  Shape: {self.df_original.shape}")
            print(f"  Memória: {self.df_original.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

            self._registrar_etapa("Carregamento", {
                'linhas': len(self.df_original),
                'colunas': len(self.df_original.columns),
                'memoria_mb': self.df_original.memory_usage(deep=True).sum() / 1024**2
            })

            return self

        except Exception as e:
            print(f"✗ Erro ao carregar dados: {str(e)}")
            raise

    def explorar_dados(self):
        """
        Realiza exploração inicial dos dados
        """
        print(f"\n{'='*70}")
        print("ETAPA 1: EXPLORAÇÃO DE DADOS")
        print(f"{'='*70}\n")

        # Informações básicas
        print("📊 Informações Básicas:")
        print(f"  Linhas: {len(self.df_processado):,}")
        print(f"  Colunas: {len(self.df_processado.columns)}")
        print(f"  Colunas: {list(self.df_processado.columns)}")

        # Tipos de dados
        print(f"\n📋 Tipos de Dados:")
        for dtype, count in self.df_processado.dtypes.value_counts().items():
            print(f"  {dtype}: {count} colunas")

        # Valores ausentes
        missing_total = self.df_processado.isnull().sum().sum()
        missing_percent = 100 * missing_total / (len(self.df_processado) * len(self.df_processado.columns))

        print(f"\n❓ Valores Ausentes:")
        print(f"  Total: {missing_total:,} ({missing_percent:.2f}%)")

        if missing_total > 0:
            print(f"  Por coluna:")
            for col in self.df_processado.columns:
                missing = self.df_processado[col].isnull().sum()
                if missing > 0:
                    print(f"    {col}: {missing} ({100*missing/len(self.df_processado):.2f}%)")

        # Duplicatas
        duplicatas = self.df_processado.duplicated().sum()
        print(f"\n🔄 Duplicatas: {duplicatas} ({100*duplicatas/len(self.df_processado):.2f}%)")

        self._registrar_etapa("Exploração", {
            'missing_total': int(missing_total),
            'missing_percent': float(missing_percent),
            'duplicatas': int(duplicatas)
        })

        return self

    def limpar_dados(self, config):
        """
        Aplica limpeza de dados baseada em configuração

        config = {
            'remover_duplicatas': True,
            'tratar_missing': {
                'colunas_remover': ['col1'],
                'colunas_imputar_media': ['col2'],
                'colunas_imputar_mediana': ['col3'],
                'colunas_imputar_moda': ['col4']
            },
            'remover_outliers': {
                'metodo': 'iqr',  # 'iqr' ou 'zscore'
                'colunas': ['col5']
            },
            'padronizar_texto': {
                'colunas': ['col6'],
                'metodo': 'title'  # 'lower', 'upper', 'title'
            }
        }
        """
        print(f"\n{'='*70}")
        print("ETAPA 2: LIMPEZA DE DADOS")
        print(f"{'='*70}\n")

        linhas_antes = len(self.df_processado)
        acoes_realizadas = []

        # 1. Remover duplicatas
        if config.get('remover_duplicatas', False):
            duplicatas_antes = self.df_processado.duplicated().sum()
            self.df_processado = self.df_processado.drop_duplicates()
            duplicatas_removidas = duplicatas_antes
            print(f"✓ Removidas {duplicatas_removidas} linhas duplicadas")
            acoes_realizadas.append(f"Duplicatas removidas: {duplicatas_removidas}")

        # 2. Tratar valores ausentes
        tratar_missing = config.get('tratar_missing', {})

        # Remover colunas
        cols_remover = tratar_missing.get('colunas_remover', [])
        if cols_remover:
            self.df_processado = self.df_processado.drop(columns=cols_remover, errors='ignore')
            print(f"✓ Removidas colunas: {cols_remover}")
            acoes_realizadas.append(f"Colunas removidas: {len(cols_remover)}")

        # Imputar média
        cols_media = tratar_missing.get('colunas_imputar_media', [])
        for col in cols_media:
            if col in self.df_processado.columns:
                missing_antes = self.df_processado[col].isnull().sum()
                media = self.df_processado[col].mean()
                self.df_processado[col].fillna(media, inplace=True)
                print(f"✓ {col}: imputados {missing_antes} valores com média {media:.2f}")
                acoes_realizadas.append(f"{col}: média")

        # Imputar mediana
        cols_mediana = tratar_missing.get('colunas_imputar_mediana', [])
        for col in cols_mediana:
            if col in self.df_processado.columns:
                missing_antes = self.df_processado[col].isnull().sum()
                mediana = self.df_processado[col].median()
                self.df_processado[col].fillna(mediana, inplace=True)
                print(f"✓ {col}: imputados {missing_antes} valores com mediana {mediana:.2f}")
                acoes_realizadas.append(f"{col}: mediana")

        # Imputar moda
        cols_moda = tratar_missing.get('colunas_imputar_moda', [])
        for col in cols_moda:
            if col in self.df_processado.columns:
                missing_antes = self.df_processado[col].isnull().sum()
                moda = self.df_processado[col].mode()[0] if not self.df_processado[col].mode().empty else None
                if moda is not None:
                    self.df_processado[col].fillna(moda, inplace=True)
                    print(f"✓ {col}: imputados {missing_antes} valores com moda '{moda}'")
                    acoes_realizadas.append(f"{col}: moda")

        # 3. Remover outliers
        remover_outliers = config.get('remover_outliers', {})
        if remover_outliers:
            metodo = remover_outliers.get('metodo', 'iqr')
            colunas = remover_outliers.get('colunas', [])

            for col in colunas:
                if col not in self.df_processado.columns:
                    continue

                linhas_antes_outlier = len(self.df_processado)

                if metodo == 'iqr':
                    Q1 = self.df_processado[col].quantile(0.25)
                    Q3 = self.df_processado[col].quantile(0.75)
                    IQR = Q3 - Q1
                    lower_bound = Q1 - 1.5 * IQR
                    upper_bound = Q3 + 1.5 * IQR

                    self.df_processado = self.df_processado[
                        (self.df_processado[col] >= lower_bound) &
                        (self.df_processado[col] <= upper_bound)
                    ]

                outliers_removidos = linhas_antes_outlier - len(self.df_processado)
                if outliers_removidos > 0:
                    print(f"✓ {col}: removidos {outliers_removidos} outliers ({metodo})")
                    acoes_realizadas.append(f"{col}: {outliers_removidos} outliers")

        # 4. Padronizar texto
        padronizar = config.get('padronizar_texto', {})
        if padronizar:
            colunas = padronizar.get('colunas', [])
            metodo = padronizar.get('metodo', 'title')

            for col in colunas:
                if col in self.df_processado.columns:
                    if metodo == 'lower':
                        self.df_processado[col] = self.df_processado[col].str.lower()
                    elif metodo == 'upper':
                        self.df_processado[col] = self.df_processado[col].str.upper()
                    elif metodo == 'title':
                        self.df_processado[col] = self.df_processado[col].str.title()

                    self.df_processado[col] = self.df_processado[col].str.strip()
                    print(f"✓ {col}: texto padronizado ({metodo})")
                    acoes_realizadas.append(f"{col}: padronizado")

        linhas_depois = len(self.df_processado)
        linhas_removidas = linhas_antes - linhas_depois

        print(f"\n📊 Resumo da Limpeza:")
        print(f"  Linhas removidas: {linhas_removidas}")
        print(f"  Linhas restantes: {linhas_depois}")

        self._registrar_etapa("Limpeza", {
            'linhas_removidas': int(linhas_removidas),
            'acoes': acoes_realizadas
        })

        return self

    def transformar_dados(self, config):
        """
        Aplica transformações nos dados

        config = {
            'converter_tipos': {
                'data_venda': 'datetime'
            },
            'criar_features_temporais': ['data_venda'],
            'normalizar': {
                'metodo': 'standardization',  # ou 'minmax'
                'colunas': ['preco', 'quantidade']
            },
            'encoding': {
                'onehot': ['categoria'],
                'label': ['status']
            }
        }
        """
        print(f"\n{'='*70}")
        print("ETAPA 3: TRANSFORMAÇÃO DE DADOS")
        print(f"{'='*70}\n")

        colunas_antes = len(self.df_processado.columns)
        transformacoes = []

        # 1. Converter tipos
        converter_tipos = config.get('converter_tipos', {})
        for col, tipo in converter_tipos.items():
            if col in self.df_processado.columns:
                try:
                    if tipo == 'datetime':
                        self.df_processado[col] = pd.to_datetime(self.df_processado[col])
                    elif tipo == 'int':
                        self.df_processado[col] = pd.to_numeric(self.df_processado[col], errors='coerce').astype('Int64')
                    elif tipo == 'float':
                        self.df_processado[col] = pd.to_numeric(self.df_processado[col], errors='coerce')

                    print(f"✓ {col}: convertido para {tipo}")
                    transformacoes.append(f"{col}: {tipo}")
                except Exception as e:
                    print(f"✗ Erro ao converter {col}: {str(e)}")

        # 2. Features temporais
        cols_temporais = config.get('criar_features_temporais', [])
        for col in cols_temporais:
            if col in self.df_processado.columns:
                self.df_processado[f'{col}_ano'] = self.df_processado[col].dt.year
                self.df_processado[f'{col}_mes'] = self.df_processado[col].dt.month
                self.df_processado[f'{col}_dia_semana'] = self.df_processado[col].dt.dayofweek
                print(f"✓ {col}: features temporais criadas (ano, mes, dia_semana)")
                transformacoes.append(f"{col}: features temporais")

        # 3. Normalização
        normalizar_config = config.get('normalizar', {})
        if normalizar_config:
            metodo = normalizar_config.get('metodo', 'standardization')
            colunas = normalizar_config.get('colunas', [])

            for col in colunas:
                if col in self.df_processado.columns:
                    if metodo == 'standardization':
                        mean = self.df_processado[col].mean()
                        std = self.df_processado[col].std()
                        self.df_processado[f'{col}_std'] = (self.df_processado[col] - mean) / std
                        print(f"✓ {col}: standardization aplicado")
                    elif metodo == 'minmax':
                        min_val = self.df_processado[col].min()
                        max_val = self.df_processado[col].max()
                        self.df_processado[f'{col}_minmax'] = (self.df_processado[col] - min_val) / (max_val - min_val)
                        print(f"✓ {col}: min-max scaling aplicado")

                    transformacoes.append(f"{col}: {metodo}")

        # 4. Encoding
        encoding_config = config.get('encoding', {})

        # One-Hot Encoding
        cols_onehot = encoding_config.get('onehot', [])
        for col in cols_onehot:
            if col in self.df_processado.columns:
                dummies = pd.get_dummies(self.df_processado[col], prefix=col, drop_first=True)
                self.df_processado = pd.concat([self.df_processado, dummies], axis=1)
                print(f"✓ {col}: one-hot encoding ({len(dummies.columns)} colunas criadas)")
                transformacoes.append(f"{col}: one-hot")

        # Label Encoding
        cols_label = encoding_config.get('label', [])
        for col in cols_label:
            if col in self.df_processado.columns:
                self.df_processado[f'{col}_label'] = pd.Categorical(self.df_processado[col]).codes
                print(f"✓ {col}: label encoding aplicado")
                transformacoes.append(f"{col}: label")

        colunas_depois = len(self.df_processado.columns)
        colunas_criadas = colunas_depois - colunas_antes

        print(f"\n📊 Resumo da Transformação:")
        print(f"  Colunas criadas: {colunas_criadas}")
        print(f"  Total de colunas: {colunas_depois}")

        self._registrar_etapa("Transformação", {
            'colunas_criadas': int(colunas_criadas),
            'transformacoes': transformacoes
        })

        return self

    def validar_dados(self, regras=None):
        """
        Valida dados processados
        """
        print(f"\n{'='*70}")
        print("ETAPA 4: VALIDAÇÃO DE DADOS")
        print(f"{'='*70}\n")

        validacoes = {
            'passou': 0,
            'falhou': 0,
            'detalhes': []
        }

        # Validações padrão
        # 1. Sem valores ausentes críticos
        missing = self.df_processado.isnull().sum().sum()
        if missing == 0:
            print("✓ Nenhum valor ausente")
            validacoes['passou'] += 1
        else:
            print(f"⚠ {missing} valores ausentes restantes")
            validacoes['detalhes'].append(f"{missing} valores ausentes")

        # 2. Sem duplicatas
        duplicatas = self.df_processado.duplicated().sum()
        if duplicatas == 0:
            print("✓ Nenhuma duplicata")
            validacoes['passou'] += 1
        else:
            print(f"✗ {duplicatas} duplicatas encontradas")
            validacoes['falhou'] += 1
            validacoes['detalhes'].append(f"{duplicatas} duplicatas")

        # 3. Tamanho do dataset
        if len(self.df_processado) > 0:
            print(f"✓ Dataset contém {len(self.df_processado)} registros")
            validacoes['passou'] += 1
        else:
            print("✗ Dataset vazio")
            validacoes['falhou'] += 1

        # Validações customizadas
        if regras:
            for regra_nome, regra_func in regras.items():
                try:
                    resultado = regra_func(self.df_processado)
                    if resultado:
                        print(f"✓ {regra_nome}")
                        validacoes['passou'] += 1
                    else:
                        print(f"✗ {regra_nome}")
                        validacoes['falhou'] += 1
                except Exception as e:
                    print(f"✗ {regra_nome}: erro - {str(e)}")
                    validacoes['falhou'] += 1

        print(f"\n📊 Resumo da Validação:")
        print(f"  Passou: {validacoes['passou']}")
        print(f"  Falhou: {validacoes['falhou']}")

        self._registrar_etapa("Validação", validacoes)

        return self

    def salvar_dados(self, caminho_saida, formato='csv'):
        """
        Salva dados processados
        """
        print(f"\n{'='*70}")
        print("SALVANDO DADOS PROCESSADOS")
        print(f"{'='*70}\n")

        try:
            if formato == 'csv':
                self.df_processado.to_csv(caminho_saida, index=False)
            elif formato == 'parquet':
                self.df_processado.to_parquet(caminho_saida, index=False)
            elif formato == 'excel':
                self.df_processado.to_excel(caminho_saida, index=False)

            tamanho_mb = os.path.getsize(caminho_saida) / 1024**2
            print(f"✓ Dados salvos em: {caminho_saida}")
            print(f"  Formato: {formato}")
            print(f"  Tamanho: {tamanho_mb:.2f} MB")

            self._registrar_etapa("Salvamento", {
                'arquivo': caminho_saida,
                'formato': formato,
                'tamanho_mb': float(tamanho_mb)
            })

        except Exception as e:
            print(f"✗ Erro ao salvar: {str(e)}")

    def gerar_relatorio(self):
        """
        Gera relatório completo do pipeline
        """
        self.tempo_fim = datetime.now()
        tempo_total = (self.tempo_fim - self.tempo_inicio).total_seconds()

        print(f"\n{'='*70}")
        print("RELATÓRIO FINAL DO PIPELINE")
        print(f"{'='*70}\n")

        print(f"Pipeline: {self.nome_pipeline}")
        print(f"Início: {self.tempo_inicio.strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"Fim: {self.tempo_fim.strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"Duração: {tempo_total:.2f} segundos")

        print(f"\n📊 Comparação Antes/Depois:")
        print(f"  Linhas: {len(self.df_original):,} → {len(self.df_processado):,}")
        print(f"  Colunas: {len(self.df_original.columns)} → {len(self.df_processado.columns)}")

        mem_antes = self.df_original.memory_usage(deep=True).sum() / 1024**2
        mem_depois = self.df_processado.memory_usage(deep=True).sum() / 1024**2
        print(f"  Memória: {mem_antes:.2f} MB → {mem_depois:.2f} MB")

        missing_antes = self.df_original.isnull().sum().sum()
        missing_depois = self.df_processado.isnull().sum().sum()
        print(f"  Missing: {missing_antes:,} → {missing_depois:,}")

        print(f"\n🔄 Etapas Executadas:")
        for etapa in self.etapas_executadas:
            print(f"  • {etapa['nome']}")

        # Salvar relatório JSON
        relatorio = {
            'pipeline': self.nome_pipeline,
            'inicio': self.tempo_inicio.strftime('%Y-%m-%d %H:%M:%S'),
            'fim': self.tempo_fim.strftime('%Y-%m-%d %H:%M:%S'),
            'duracao_segundos': tempo_total,
            'dados_originais': {
                'linhas': len(self.df_original),
                'colunas': len(self.df_original.columns),
                'memoria_mb': mem_antes,
                'missing': int(missing_antes)
            },
            'dados_processados': {
                'linhas': len(self.df_processado),
                'colunas': len(self.df_processado.columns),
                'memoria_mb': mem_depois,
                'missing': int(missing_depois)
            },
            'etapas': self.etapas_executadas
        }

        with open(f'relatorio_{self.nome_pipeline}.json', 'w', encoding='utf-8') as f:
            json.dump(relatorio, f, indent=2, ensure_ascii=False)

        print(f"\n✓ Relatório JSON salvo: relatorio_{self.nome_pipeline}.json")

        return relatorio

    def _registrar_etapa(self, nome, metricas):
        """Registra etapa executada"""
        self.etapas_executadas.append({
            'nome': nome,
            'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'metricas': metricas
        })

    def obter_dados_processados(self):
        """Retorna dataframe processado"""
        return self.df_processado


# ==================== CONFIGURAÇÕES DE EXEMPLO ====================

CONFIG_LIMPEZA_VENDAS = {
    'remover_duplicatas': True,
    'tratar_missing': {
        'colunas_imputar_mediana': ['quantidade'],
        'colunas_imputar_media': ['preco_unitario'],
        'colunas_imputar_moda': ['status']
    },
    'remover_outliers': {
        'metodo': 'iqr',
        'colunas': ['preco_unitario']
    },
    'padronizar_texto': {
        'colunas': ['produto', 'cliente', 'status'],
        'metodo': 'title'
    }
}

CONFIG_TRANSFORMACAO_VENDAS = {
    'converter_tipos': {
        'data_venda': 'datetime',
        'quantidade': 'int'
    },
    'criar_features_temporais': ['data_venda'],
    'normalizar': {
        'metodo': 'standardization',
        'colunas': ['preco_unitario', 'quantidade']
    },
    'encoding': {
        'onehot': ['produto'],
        'label': ['status']
    }
}


# ==================== EXEMPLO DE USO ====================

def executar_pipeline_vendas():
    """
    Executa pipeline completo para dados de vendas
    """
    # Criar pipeline
    pipeline = PipelinePreProcessamento(nome_pipeline="Vendas_V1")

    # Executar etapas
    (pipeline
        .carregar_dados('dados_vendas_raw.csv')
        .explorar_dados()
        .limpar_dados(CONFIG_LIMPEZA_VENDAS)
        .transformar_dados(CONFIG_TRANSFORMACAO_VENDAS)
        .validar_dados()
        .salvar_dados('dados_vendas_processados.csv', formato='csv')
        .gerar_relatorio()
    )

    return pipeline


def executar_pipeline_customizado():
    """
    Exemplo de pipeline com configurações customizadas
    """
    # Configuração customizada
    config_limpeza = {
        'remover_duplicatas': True,
        'tratar_missing': {
            'colunas_imputar_mediana': ['valor'],
        },
        'padronizar_texto': {
            'colunas': ['localizacao'],
            'metodo': 'title'
        }
    }

    config_transformacao = {
        'converter_tipos': {
            'timestamp': 'datetime'
        },
        'criar_features_temporais': ['timestamp']
    }

    # Regras de validação customizadas
    def validar_valores_positivos(df):
        return (df['valor'] > 0).all()

    def validar_sensores_unicos(df):
        return df['sensor_id'].nunique() > 0

    regras_validacao = {
        'Valores positivos': validar_valores_positivos,
        'Sensores únicos': validar_sensores_unicos
    }

    # Executar pipeline
    pipeline = PipelinePreProcessamento(nome_pipeline="Sensores_V1")

    (pipeline
        .carregar_dados('dados_sensores_raw.csv')
        .explorar_dados()
        .limpar_dados(config_limpeza)
        .transformar_dados(config_transformacao)
        .validar_dados(regras_validacao)
        .salvar_dados('dados_sensores_processados.csv')
        .gerar_relatorio()
    )

    return pipeline


if __name__ == "__main__":
    print("""
╔══════════════════════════════════════════════════════════════════════╗
║                PIPELINE DE PRÉ-PROCESSAMENTO DE DADOS                ║
║                          Aula 02 - PÓS-GRADUAÇÃO                     ║
╚══════════════════════════════════════════════════════════════════════╝
    """)

    # Verificar se arquivos existem
    if not os.path.exists('dados_vendas_raw.csv'):
        print("⚠ Execute primeiro o script 01_geracao_dados_exemplo.py")
        print("   para criar os arquivos de dados necessários.\n")
        exit(1)

    print("\n🚀 Executando Pipeline de Vendas...")
    print("="*70)

    pipeline_vendas = executar_pipeline_vendas()

    print("\n\n" + "="*70)
    print("✓ PIPELINE CONCLUÍDO COM SUCESSO!")
    print("="*70)

    print("\n📁 Arquivos Gerados:")
    print("  • dados_vendas_processados.csv")
    print("  • relatorio_Vendas_V1.json")

    # Executar pipeline de sensores se arquivo existir
    if os.path.exists('dados_sensores_raw.csv'):
        print("\n\n🚀 Executando Pipeline de Sensores...")
        print("="*70)
        pipeline_sensores = executar_pipeline_customizado()
        print("\n📁 Arquivos Adicionais Gerados:")
        print("  • dados_sensores_processados.csv")
        print("  • relatorio_Sensores_V1.json")

    print("\n\n" + "="*70)
    print("💡 DICAS PARA OS ALUNOS:")
    print("="*70)
    print("""
1. Explore os relatórios JSON gerados para entender cada etapa
2. Modifique as configurações de limpeza e transformação
3. Adicione suas próprias regras de validação
4. Teste o pipeline com diferentes datasets
5. Compare os dados antes e depois do processamento
6. Experimente diferentes métodos de imputação e normalização
    """)

    print("\n✓ Exercícios práticos prontos para a aula!")
    print("="*70)